In [2]:
!pip install git+https://github.com/shk06/FlirImageExtractor.git

  Cloning https://github.com/shk06/FlirImageExtractor.git to c:\nak-temp\pip-req-build-haf_wr1f
  Resolved https://github.com/shk06/FlirImageExtractor.git to commit 13549d460c62275d3b694ce354dce8adf3c38408
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/shk06/FlirImageExtractor.git 'C:\NAK-TEMP\pip-req-build-haf_wr1f'

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
# Force OpenBLAS to use only one thread.
os.environ["OPENBLAS_NUM_THREADS"] = "1"

import contextlib
import threadpoolctl

# Monkey-patch threadpool_limits to bypass the actual thread limiting.
@contextlib.contextmanager
def dummy_threadpool_limits(*args, **kwargs):
    yield

threadpoolctl.threadpool_limits = dummy_threadpool_limits

# Importing Functions
import flirimageextractor
import numpy as np
import subprocess
import cv2
import glob
import matplotlib
matplotlib.use('TKAgg') # Needed to have figures display properly.
import matplotlib.pyplot as plt
from matplotlib import colors

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [4]:
from PIL import Image
import seaborn as sns

C:\Users\alpha\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [5]:
def save_thermal_csv(flirobj, filename):
    """
    Function that saves the numpy array as a .csv

    INPUTS:
    1) flirobj: the flirimageextractor object.
    2) filename: a string containing the location of the output csv file.

    OUTPUTS:
    Saves a csv of the thermal image where each value is a pixel in the thermal image.
    """
    data = flirobj.get_thermal_np()
    np.savetxt(filename, data, delimiter=',')

In [6]:
def save_segment_csv(flirobj, filename, mask, emiss=[0]):
    """
    Function that saves the segment numpy array as a .csv

    INPUTS:
    1) flirobj: the flirimageextractor object.
    2) filename: a string containing the location of the output csv file.
    3) mask: a binary mask with class pixels set as 1.
    4) emiss: OPTIONAL, a 2D numpy array with each pixel containing correct emissivity
            If provided the temperature will be corrected for emissivity
    5) plot: a flag that determine if a figure of tempeature image is displayed.
            1 = plot displayed, 0 = no plot is displayed
            
    OUTPUTS:
    Saves a csv of the thermal image where each value is a pixel in the thermal image for the unique surface in question.
    """
    if len(emiss) == 1:
        therm = flirobj.get_thermal_np()
    else:
        therm = correct_temp_emiss(flirobj, emiss, plot=0)

    therm_masked = np.ma.masked_where(mask != 1, therm)
        
    np.savetxt(filename, data, delimiter=',')

In [7]:
def extract_coarse_image(flirobj, offset=[0], plot=1):
    """
    Function that creates the coarse RGB image that matches the resolution of the thermal image.

    INPUTS:
        1) flirobj: the flirimageextractor object.
        2) offset: optional variable that shifts the RGB image to match the same field of view as thermal image.
                If not provided the offset values will be extracted from FLIR file.
                Use the manual_img_registration function to determine offset.
        3) plot: a flag that determine if a figure of thermal and coarse cropped RGB is displayed.
                1 = plot displayed, 0 = no plot is displayed
    OUTPUTS:
        1) lowres: a 3D numpy array of RGB image that matches resolution of thermal image (It has not been cropped)
        2) crop: a 3D numpy arary of RGB image that matches resolution and field of view of thermal image.
    """
    # Get RGB Image
    visual = flirobj.rgb_image_np
    highres_ht = visual.shape[0]
    highres_wd = visual.shape[1]

    # Getting Values for Offset
    if len(offset) < 2:
        offsetx = int(subprocess.check_output([flirobj.exiftool_path, "-OffsetX", "-b", flirobj.flir_img_filename]))
        offsety = int(subprocess.check_output([flirobj.exiftool_path, "-OffsetY", "-b", flirobj.flir_img_filename]))
    else:
        offsetx = offset[0]
        offsety = offset[1]
    pipx2 = int(subprocess.check_output([flirobj.exiftool_path, "-PiPX2", "-b", flirobj.flir_img_filename])) # Width
    pipy2 = int(subprocess.check_output([flirobj.exiftool_path, "-PiPY2", "-b", flirobj.flir_img_filename])) # Height
    real2ir = float(subprocess.check_output([flirobj.exiftool_path, "-Real2IR", "-b", flirobj.flir_img_filename])) # conversion of RGB to Temp

    # Set up Arrays
    height_range = np.arange(0,highres_ht,real2ir).astype(int)
    width_range = np.arange(0,highres_wd,real2ir).astype(int)
    htv, wdv = np.meshgrid(height_range,width_range)

    # Assigning low resolution data
    lowres = np.swapaxes(visual[htv, wdv,  :], 0, 1)

    # Cropping low resolution data
    height_range = np.arange(-offsety,-offsety+pipy2).astype(int)
    width_range = np.arange(-offsetx,-offsetx+pipx2).astype(int)
    xv, yv = np.meshgrid(height_range,width_range)
    crop = np.swapaxes(lowres[xv, yv, :],0,1)

    if plot == 1:
        therm = flirobj.get_thermal_np()
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        plt.imshow(therm, cmap='jet')
        plt.title('Thermal Image')
        plt.subplot(1,2,2)
        plt.imshow(crop)
        plt.title('RGB Cropped Image')
        plt.show(block='TRUE')

    return lowres, crop

In [8]:
def manual_img_registration(flirobj):
    """
    Function that displays the thermal and RGB image so that similar locations
    can be selected in both images. It is recommended that at least three tied-points
    are collected. Using the tie points the average x and y pixel offset will be determined.

    HOW TO:
    Left click adds points, right click removes points (necessary after a pan or zoom),
    and middle click stops point collection.
    The keyboard can also be used to select points in case your mouse does not have one or
    more of the buttons. The delete and backspace keys act like right clicking
    (i.e., remove last point), the enter key terminates input and any other key
    (not already used by the window manager) selects a point.
    ESC will delete all points - do not use.

    INPUTS:
        1) flirobj: the flirimageextractor object.
    OUTPUTS:
        1) offset: a numpy array with [x pixel offset, y pixel offset] between thermal and rgb image
        2) pts_therm: a numpy array containing the image registration points for the thermal image.
        3) pts_rgb: a numpy array containing the coordinates of RGB image matching the thermal image.
    """
    # Getting Images
    therm = flirobj.get_thermal_np()
    rgb, junk = extract_coarse_image(flirobj)

    # Plot Images
    fig = plt.figure(figsize=(10,5))
    ax1 = fig.add_subplot(1,2,1)
    ax1.imshow(therm, cmap='jet')
    ax1.set_title('Thermal')
    ax1.text(0,-100,'Collect points matching features between images. Select location on thermal then RGB image.')
    ax1.text(0,-75,'Right click adds a point. Left click removes most recently added point. Middle click (or enter) stops point collection.')
    ax1.text(0,-50,'Zoom/Pan add a point, but you can remove with left click. Or use back arrow to get back to original view.')
    ax2 = fig.add_subplot(1,2,2)
    ax2.imshow(rgb)
    ax2.set_title('RGB')
    fig.subplots_adjust(left=0.05, top = 0.8, bottom=0.01,right=0.95)

    # Getting points
    pts = np.asarray(fig.ginput(-1, timeout=-1))
    idx_therm = np.arange(0,pts.shape[0],2)
    pts_therm = pts[idx_therm,:]
    idx_rgb = np.arange(1,pts.shape[0],2)
    pts_rgb = pts[idx_rgb,:]

    # Getting Difference between images to determine offset
    size_therm = pts_therm.shape[0]
    size_rgb = pts_rgb.shape[0]
    offset = [0,0]
    if size_therm == size_rgb:
        pts_diff = pts_therm - pts_rgb
        offset = np.around(np.mean(pts_diff, axis=0))
    else:
        print('Number of points do not match between images')

    plt.close()

    return offset, pts_therm, pts_rgb

In [9]:
def classify_rgb(img, K=3, plot=1):
    """
    This classifies an RGB image using K-Means clustering.
    Note: only 10 colors are specified, so will have plotting error with K > 10
    INPUTS:
        1) img: a 3D numpy array of rgb image
        2) K: optional, the number of K-Means Clusters
        3) plot: a flag that determine if multiple figures of classified is displayed.
                1 = plot displayed, 0 = no plot is displayed
    OUTPUTS:
        1) label_image: a 2D numpy array the same x an y dimensions as input rgb image,
            but each pixel is a k-means class.
        2) result_image: a 3D numpy array the same dimensions as input rgb image,
            but having undergone Color Quantization which is the process of
            reducing number of colors in an image.
    """
    # Preparing RGB Image
    vectorized = img.reshape((-1,3))
    vectorized = np.float32(vectorized)

    # K-Means
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    attempts = 10
    ret,label,center=cv2.kmeans(vectorized,K,None,criteria,attempts,cv2.KMEANS_RANDOM_CENTERS)

    # Use if you want to have quantized imaged
    center = np.uint8(center)
    res = center[label.flatten()]
    result_image = res.reshape((img.shape))

    # Labeled class image
    label_image = label.reshape((img.shape[0], img.shape[1]))

    if plot == 1:
        # Plotting Results
        coloroptions = ['b','g','r','c','m','y','k','orange','navy','gray']
        fig = plt.figure(figsize=(10,5))
        ax1 = fig.add_subplot(1,2,1)
        ax1.imshow(img)
        ax1.set_title('Original Image')
        ax1.set_xticks([])
        ax1.set_yticks([])
        ax2 = fig.add_subplot(1,2,2)
        cmap = colors.ListedColormap(coloroptions[0:K])
        ax2.imshow(label_image, cmap=cmap)
        ax2.set_title('K-Means Classes')
        ax2.set_xticks([])
        ax2.set_yticks([])
        fig.subplots_adjust(left=0.05, top = 0.8, bottom=0.01, wspace=0.05)
        plt.show(block='TRUE')

        # Plotting just K-Means with label
        ticklabels = ['1','2','3','4','5','6','7','8','9','10']
        fig, ax = plt.subplots(figsize=(5,5))
        im = ax.imshow(label_image, cmap=cmap)
        cbar = fig.colorbar(im, ax=ax, shrink = 0.6, ticks=np.arange(0,K))
        cbar.ax.set_yticklabels(ticklabels[0:K])
        cbar.ax.set_ylabel('Classes')
        plt.show(block='TRUE')

    return label_image, result_image

In [10]:
def apply_mask_to_rgb(mask, rgbimg, plot=1):
    """
    Function that applies mask to provided RGB image and returns RGB image with
    only pixels where mask is 1 and all other pixels are black. This function
    is useful to use BEFORE K-Means classification.
    INPUTS:
        1) mask: a numpy binary mask that same shape as rgbimg variable.
                0's are pixels NOT of interest and will be masked out.
                1's are pixels of interest and will be returned.
        2) rgbimg: a 3D numpy array that contains RGB image.
        3) plot: a flag that determine if a figure of masked image is displayed.
                1 = plot displayed, 0 = no plot is displayed
    OUTPUTS:
        1) masked_rgb: a 3D numpy array that contains RGB image with all pixels
                designated as 0 in the mask are black.
    """
    masked_rgb = np.zeros((rgbimg.shape[0], rgbimg.shape[1], rgbimg.shape[2]),int)
    for d in range(0,rgbimg.shape[2]):
        masked_rgb[:,:,d] = rgbimg[:,:,d] * mask

    if plot == 1:
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        plt.imshow(rgbimg)
        plt.title('Original Image')
        plt.subplot(1,2,2)
        plt.imshow(masked_rgb)
        plt.title('Masked Image')
        plt.show(block='TRUE')

    return masked_rgb

In [11]:
def create_class_mask(classimg, classinterest, plot=1):
    """
    This function creates a mask that turns all K-Means classes NOT of interest
    as 0 and all classes of interest to 1. This can be used to extract temperatures
    only for classes of interest.
    INPUTS:
        1) classinterest: a array containing the class or classes of interest.
                All other classes will be masked out
        2) classimg: the K-Means class image which is (2D) numpy array
        3) plot: a flag that determine if a figure of masked K-Means class image is displayed.
                1 = plot displayed, 0 = no plot is displayed
    OUTPUTS:
        1) mask: a 2D numpy binary mask that same shape as the first two dimensions of rgbimg variable.
                0's are pixels NOT of interest and will be masked out.
                1's are pixels of interest and will be returned.
    """
    mask = np.zeros((classimg.shape[0], classimg.shape[1]))
    if isinstance(classinterest,int):
        endrange = 1
    else:
        endrange = len(classinterest)

    for c in range(0,endrange):
        idx_x, idx_y = np.where(classimg == c)
        mask[idx_x, idx_y] = 1

    if plot == 1:
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        plt.imshow(classimg)
        plt.title('Original K-Means Classes')
        plt.subplot(1,2,2)
        plt.imshow(mask, cmap='gray')
        plt.title('Masked Classes')
        plt.show(block='TRUE')

    return mask

In [12]:
def extract_temp_for_class(flirobj, mask, emiss=[0], plot=1):
    """
    This function creates a numpy array thermal image that ONLY contains pixels for class
    of interest with all other pixels set to 0. This is for a SINGLE image
    INPUTS:
        1) flirobj: the flirimageextractor object
        2) mask: a binary mask with class pixels set as 1.
        3) emiss: OPTIONAL, a 2D numpy array with each pixel containing correct emissivity
                If provided the temperature will be corrected for emissivity
        4) plot: a flag that determine if a figure of tempeature image is displayed.
                1 = plot displayed, 0 = no plot is displayed
    OUTPUTS:
        1) therm_masked: a 2D numpy array of temperature values for a class of interest
    """
    if len(emiss) == 1:
        therm = flirobj.get_thermal_np()
    else:
        therm = correct_temp_emiss(flirobj, emiss, plot=0)

    therm_masked = np.ma.masked_where(mask != 1, therm)

    if plot == 1:
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        plt.imshow(therm, cmap='jet')
        plt.subplot(1,2,2)
        plt.imshow(therm_masked, cmap='jet')
        plt.show(block='TRUE')

    return therm_masked

In [13]:
def batch_extract_temp_for_class(dirLoc, mask, emiss=[0], exiftoolpath=''):
    """
    This function creates a 3D numpy array thermal image that ONLY contains pixels for class
    of interest with all other pixels set to 0. This is for a directory of images.
    INPUTS:
        1) dirLoc: a string containing the directory of FLIR images.
        2) mask: a binary mask with class pixels set as 1.
        3) emiss: a 2D numpy array with each pixel containing correct emissivity
        4) exiftoolpath = OPTIONAL a string containing the location of exiftools.
                Only use if the exiftool path is different than the python path.
    OUTPUTS:
        1) all_temp: a 3D numpy array of temperature values for a class of interest
                The first two dimensions will match the dimensions of a single temperature image.
                The third dimension size will match the number of files in directory.
    """
    filelist = glob.glob(dirLoc + '*')
    print('Found ' + str(len(filelist)) + ' files.')
    all_temp = np.ma.empty((mask.shape[0], mask.shape[1], len(filelist)))

    for f in range(0,len(filelist)):
        # Get individual file
        if not exiftoolpath:
            flir = flirimageextractor.FlirImageExtractor()
        else:
            flir = flirimageextractor.FlirImageExtractor(exiftool_path=exiftoolpath)

        flir.process_image(filelist[f], RGB=True)

        if len(emiss) == 1:
            all_temp[:,:,f] = extract_temp_for_class(flir, mask, plot=0)
        else:
            all_temp[:,:,f] = extract_temp_for_class(flir, mask, emiss, plot=0)

    return all_temp

In [14]:
def plot_temp_timeseries(temp):
    """
    Function that plots the mean, min, and max temperature for a temperature timeseries.
    INPUTS:
        1) temp: a 3D numpy array of temperature values for a class of interest
                The first two dimensions will match the dimensions of a single temperature image.
                The third dimension size will match the number of files in directory.
    OUTPUTS:
        figure of mean, min, and maximum temperature across timeseries
    """
    # Setting up Variables
    minmaxtemp = np.zeros((2,temp.shape[2]))
    meantemp = np.zeros(temp.shape[2])

    # Loop through time steps
    for d in range(0, temp.shape[2]):
        minmaxtemp[0,d] = np.nanmin(temp[:,:,d])
        meantemp[d] = np.nanmean(temp[:,:,d])
        minmaxtemp[1,d] = np.nanmax(temp[:,:,d])

    difftemp = abs(minmaxtemp - meantemp)
    plt.figure(figsize=(10,7))
    plt.errorbar(np.arange(1, temp.shape[2]+1), meantemp, yerr=difftemp, linewidth=2, color='black')
    plt.gca().yaxis.grid(True)
    plt.title('Temperature through Timeseries')
    plt.xlabel('Time Steps')
    plt.ylabel('Temperature (Celsius)')
    plt.show(block='true')

In [15]:
def develop_correct_emissivity(class_img):
    """
    The thermal camera has an assume emissivity of 0.95, but many materials do
    not have that emissivity which changes the temperature retrieved. This code
    assigned the appropriate emissivity value for a pixel (user provided)
    using the K-Means classes.
    INPUTS:
        1) class_img: the K-Means class image
    OUTPUTS:
        1) emiss_img: a numpy array with same dimensions as K-Means class image,
                but every pixel has an emissivity value.
    """
    K = len(np.unique(class_img))

    # Plotting just K-Means with label
    coloroptions = ['b','g','r','c','m','y','k','orange','navy','gray']
    ticklabels = ['1','2','3','4','5','6','7','8','9','10']
    fig, ax = plt.subplots(figsize=(5,5))
    cmap = colors.ListedColormap(coloroptions[0:K])
    im = ax.imshow(class_img, cmap=cmap)
    cbar = fig.colorbar(im, ax=ax, shrink = 0.6, ticks=np.arange(0,K))
    cbar.ax.set_yticklabels(ticklabels[0:K])
    cbar.ax.set_ylabel('Classes')
    plt.show(block='true')

    print('Input the emissivity for each class. If unknown put 0.95')
    emiss = np.zeros((K))
    for c in range(0,K):
        strout = 'Emissivity for Class ' + str(c+1) + ': '
        emiss[c] = input(strout)

    emiss_img = np.zeros((class_img.shape[0], class_img.shape[1]))
    for e in range(0, K):
        idx_x, idx_y = np.where(class_img == e)
        emiss_img[idx_x, idx_y] = emiss[e]

    return emiss_img

In [16]:
def correct_temp_emiss(flirobj, emiss, plot=1):
    """
    The thermal camera has an assume emissivity of 0.95, but many materials do
    not have that emissivity which changes the temperature retrieved. This function
    takes in the user provided emissivity array for each pixel and corrects the
    temperature values.
    This uses the stephan boltzman equation.
    INPUTS:
        1) flirobj: a flirimageextractor object
        2) emiss: a 2D numpy array with each pixel containing correct emissivity
                Using these values the temperature will be corrected for emissivity
    OUTPUTS:
        1) corrected_temp: a 2D numpy array with corrected temperature values
    """
    therm = flirobj.get_thermal_np()

    sbconstant = 0.00000005670374419

    # Get total flux using the assumed emissivity of 0.95
    totalflux = 0.95 * sbconstant * np.power(therm, 4)

    # Solving for Temperature given new emissivities
    value = totalflux/(emiss*sbconstant)
    corrected_temp = np.power(value,1/4)

    if plot == 1:
        plt.figure(figsize=(5,5))
        plt.imshow(corrected_temp, cmap='jet')
        plt.colorbar()
        plt.show(block='true')

    return corrected_temp

In [17]:
filename = "C:\\Users\\alpha\\OneDrive\\Documents\\FLIR0912.jpg"
flir = flirimageextractor.FlirImageExtractor("C:\\Users\\alpha\\exiftool-12.92_64\\exiftool.exe")
flir.process_image(filename, RGB=True)

therm = flir.get_thermal_np()
rgb_fullres = flir.get_rgb_np()

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(therm)
plt.title('Thermal Image')
plt.subplot(1,2,2)
plt.imshow(rgb_fullres)
plt.title('RGB Full Resolution Image')
plt.show(block='TRUE') # I needed to have block=TRUE for image to remain displayed

In [18]:
import numpy as np
from PIL import Image

# Function to load and convert mask image to numpy array
def load_mask_image(image_path):
    # Open the image
    mask_image = Image.open(image_path).convert("L")  # Convert to grayscale
    
    # Convert the image to a numpy array
    mask_array = np.array(mask_image)
    
    # Threshold the array to create a binary mask (1s for mask, 0s for background)
    binary_mask = (mask_array > 0).astype(np.uint8)
    
    return binary_mask

# Example usage
image_path = "C:\\Users\\alpha\\Downloads\\EXAMPLE_combined_mask.png"
binary_mask = load_mask_image(image_path)

In [19]:
print(binary_mask)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]]


In [20]:
pavement_mask = extract_temp_for_class(flir, binary_mask, emiss=[0], plot=1)
print(pavement_mask)

[[-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 ...
 [24.353940220950278 24.42917332196845 24.44170713903719 ... -- -- --]
 [24.30375602448197 24.391563245467523 24.42917332196845 ... -- -- --]
 [24.266102737452854 24.353940220950278 24.391563245467523 ... -- -- --]]


In [21]:
import pandas as pd

In [22]:
# Function to load and convert mask image to numpy array
def load_mask_image(image_path):
    mask_image = Image.open(image_path).convert("L")  # Convert to grayscale
    mask_array = np.array(mask_image)
    binary_mask = (mask_array > 0).astype(np.uint8)  # Threshold to create a binary mask
    return binary_mask

In [23]:
# Function to extract temperature for class
def extract_temp_for_class(flirobj, mask, emiss=[0], plot=0):
    if len(emiss) == 1:
        therm = flirobj.get_thermal_np()
    else:
        therm = correct_temp_emiss(flirobj, emiss, plot=0)

    therm_masked = np.ma.masked_where(mask != 1, therm)

    if plot == 1:
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(therm, cmap='jet')
        plt.subplot(1, 2, 2)
        plt.imshow(therm_masked, cmap='jet')
        plt.show(block='TRUE')

    return therm_masked

In [24]:
# Function to process directories and return temperature data from each image's segments
def process_directories(thermal_dir, track_dir, platform_dir, ceiling_dir, column_dir):
    feature_dirs = {
        'TRACKS': track_dir,
        'PLATFORM': platform_dir,
        'CEILING': ceiling_dir,
        'COLUMN': column_dir
    }

    temperature_data = {i: {feature: [] for feature in feature_dirs.keys()} for i in range(1, 11)}

    for feature, dir_path in feature_dirs.items():
        print(f"Processing feature: {feature} from directory: {dir_path}")
        for file_name in os.listdir(dir_path):
            print(f"  - Processing file: {file_name}")
            set_id = file_name.split(feature)[1]
            location = int(set_id[:2])  # Extract the location number

            mask_path = os.path.join(dir_path, file_name)
            mask = load_mask_image(mask_path)

            thermal_file_name = f"FLIR{set_id}"  # Assuming thermal image names follow this pattern
            thermal_path = os.path.join(thermal_dir, thermal_file_name)

            try:
                flir = flirimageextractor.FlirImageExtractor("C:\\Users\\alpha\\exiftool-12.92_64\\exiftool.exe")
                flir.process_image(thermal_path)

                temperature_masked = extract_temp_for_class(flir, mask, plot=0)
                if temperature_masked is not None:
                    temperature_data[location][feature].append(temperature_masked.compressed())

                print(f"    - Processed {thermal_path} and {mask_path} successfully.")
            except Exception as e:
                print(f"    - Failed to process {thermal_path} or {mask_path}: {e}")

    return temperature_data

In [25]:
# Function to create a spatial series plot for multiple stations at one given time on a particular day.
# It plots all four violin plots side-by-side per station on one axes.

def create_spatial_series_plot_1(temperature_data):
    flattened_data = []
    for location, features in temperature_data.items():
        for feature, temp_lists in features.items():
            
            for temp_list in temp_lists:
                for temp in temp_list:
                    flattened_data.append((location, feature, temp))

    # Convert to DataFrame for seaborn
    df = pd.DataFrame(flattened_data, columns=['Location', 'Feature', 'Temperature'])

    plt.figure(figsize=(14, 8))

    # Plot TRACKS/PLATFORM and CEILING/COLUMN with dodge
    sns.violinplot(x='Location', y='Temperature', hue='Feature', data=df,
                   split=False, inner='quartile', dodge=True,
                   palette={'TRACKS': 'blue', 'PLATFORM': 'green', 'CEILING': 'orange', 'COLUMN': 'purple'})

    plt.title('Spatial-Series Plot of Temperature Across Different Features')
    plt.xlabel('Location')
    plt.ylabel('Temperature (°C)')
    plt.ylim(0, 44)
    plt.legend(title='Feature', loc='upper right')
    plt.xticks(rotation=45)  # Rotate x-tick labels for better readability
    plt.show()

In [26]:
# Function to create a spatial series plot for multiple stations at one given time on a particular day.
# It plots all two violin plots side-by-side on two separate axes, resulting in a total of four feature types per station.

def create_spatial_series_plot_2(temperature_data):
    flattened_data = []
    for location, features in temperature_data.items():
        for feature, temp_lists in features.items():
            for temp_list in temp_lists:
                for temp in temp_list:
                    flattened_data.append((location, feature, temp))

    # Convert to DataFrame for seaborn
    df = pd.DataFrame(flattened_data, columns=['Location', 'Feature', 'Temperature'])

    # Separate the data for each pair of features
    df_tracks_platform = df[df['Feature'].isin(['TRACKS', 'PLATFORM'])]
    df_ceiling_column = df[df['Feature'].isin(['CEILING', 'COLUMN'])]

    # Check if DataFrames are empty and handle accordingly
    if df_tracks_platform.empty:
        print("Warning: No data available for TRACKS or PLATFORM.")
        return
    if df_ceiling_column.empty:
        print("Warning: No data available for CEILING or COLUMN.")
        return

    # Create subplots
    fig, axes = plt.subplots(1, 2, figsize=(14, 8), sharey=True)

    # Plot TRACKS and PLATFORM
    sns.violinplot(ax=axes[0], x='Location', y='Temperature', hue='Feature', data=df_tracks_platform,
                   split=True, inner='quartile', palette={'TRACKS': 'blue', 'PLATFORM': 'green'})
    axes[0].set_title('Tracks and Platform')
    axes[0].set_xlabel('Location')
    axes[0].set_ylabel('Temperature (°C)')

    # Plot CEILING and COLUMN
    sns.violinplot(ax=axes[1], x='Location', y='Temperature', hue='Feature', data=df_ceiling_column,
                   split=True, inner='quartile', palette={'CEILING': 'orange', 'COLUMN': 'purple'})
    axes[1].set_title('Ceiling and Column')
    axes[1].set_xlabel('Location')

    # Adjust layout
    plt.tight_layout(pad=2)  # Add padding between plots
    plt.ylim(0, 44)
    plt.show()

In [27]:
# Function to create a spatial series plot for multiple stations at one given time on a particular day.
# It plots all four violin plots on top of each other.

def create_spatial_series_plot_3(temperature_data):
    flattened_data = []
    for location, features in temperature_data.items():
        for feature, temp_lists in features.items():
            for temp_list in temp_lists:
                for temp in temp_list:
                    flattened_data.append((location, feature, temp))

    # Convert to DataFrame for seaborn
    df = pd.DataFrame(flattened_data, columns=['Location', 'Feature', 'Temperature'])

    # Separate the data for each pair of features
    df_tracks_platform = df[df['Feature'].isin(['TRACKS', 'PLATFORM'])]
    df_ceiling_column = df[df['Feature'].isin(['CEILING', 'COLUMN'])]

    # Check if DataFrames are empty and handle accordingly
    if df_tracks_platform.empty:
        print("Warning: No data available for TRACKS or PLATFORM.")
        return
    if df_ceiling_column.empty:
        print("Warning: No data available for CEILING or COLUMN.")
        return

    plt.figure(figsize=(14, 8))

    # Plot TRACKS and PLATFORM
    sns.violinplot(x='Location', y='Temperature', hue='Feature', data=df_tracks_platform, 
                   split=True, inner='quartile', position=0, palette={'TRACKS': 'blue', 'PLATFORM': 'green'})

    # Plot CEILING and COLUMN
    sns.violinplot(x='Location', y='Temperature', hue='Feature', data=df_ceiling_column, 
                   split=True, inner='quartile', position=1, palette={'CEILING': 'orange', 'COLUMN': 'purple'})

    plt.title('Spatial-Series Plot of Temperature Across Different Features')
    plt.xlabel('Location')
    plt.ylabel('Temperature (°C)')
    plt.legend(title='Feature', loc='upper right')
    plt.ylim(0, 44)
    plt.show()

In [28]:
# Directory paths
thermal_dir = "C:\\Users\\alpha\\Downloads\\bd_7_10_2024_thermal"
track_dir = "C:\\Users\\alpha\\Downloads\\bd_7_10_2024_tracks"
platform_dir = "C:\\Users\\alpha\\Downloads\\bd_7_10_2024_platform"
ceiling_dir = "C:\\Users\\alpha\\Downloads\\bd_7_10_2024_ceilings"
column_dir = "C:\\Users\\alpha\\Downloads\\bd_7_10_2024_columns"

# Process directories and extract temperature data
temperature_data = process_directories(thermal_dir, track_dir, platform_dir, ceiling_dir, column_dir)

Processing feature: TRACKS from directory: C:\Users\alpha\Downloads\bd_7_10_2024_tracks
  - Processing file: TRACKS0101.jpg
    - Processed C:\Users\alpha\Downloads\bd_7_10_2024_thermal\FLIR0101.jpg and C:\Users\alpha\Downloads\bd_7_10_2024_tracks\TRACKS0101.jpg successfully.
  - Processing file: TRACKS0102.jpg
    - Processed C:\Users\alpha\Downloads\bd_7_10_2024_thermal\FLIR0102.jpg and C:\Users\alpha\Downloads\bd_7_10_2024_tracks\TRACKS0102.jpg successfully.
  - Processing file: TRACKS0103.jpg
    - Processed C:\Users\alpha\Downloads\bd_7_10_2024_thermal\FLIR0103.jpg and C:\Users\alpha\Downloads\bd_7_10_2024_tracks\TRACKS0103.jpg successfully.
  - Processing file: TRACKS0201.jpg
    - Processed C:\Users\alpha\Downloads\bd_7_10_2024_thermal\FLIR0201.jpg and C:\Users\alpha\Downloads\bd_7_10_2024_tracks\TRACKS0201.jpg successfully.
  - Processing file: TRACKS0202.jpg
    - Processed C:\Users\alpha\Downloads\bd_7_10_2024_thermal\FLIR0202.jpg and C:\Users\alpha\Downloads\bd_7_10_2024_tra

    - Processed C:\Users\alpha\Downloads\bd_7_10_2024_thermal\FLIR0602.jpg and C:\Users\alpha\Downloads\bd_7_10_2024_columns\COLUMN0602.jpg successfully.
  - Processing file: COLUMN0603.jpg
    - Processed C:\Users\alpha\Downloads\bd_7_10_2024_thermal\FLIR0603.jpg and C:\Users\alpha\Downloads\bd_7_10_2024_columns\COLUMN0603.jpg successfully.
  - Processing file: COLUMN0701.jpg
    - Processed C:\Users\alpha\Downloads\bd_7_10_2024_thermal\FLIR0701.jpg and C:\Users\alpha\Downloads\bd_7_10_2024_columns\COLUMN0701.jpg successfully.
  - Processing file: COLUMN0702.jpg
    - Processed C:\Users\alpha\Downloads\bd_7_10_2024_thermal\FLIR0702.jpg and C:\Users\alpha\Downloads\bd_7_10_2024_columns\COLUMN0702.jpg successfully.
  - Processing file: COLUMN0801.jpg
    - Processed C:\Users\alpha\Downloads\bd_7_10_2024_thermal\FLIR0801.jpg and C:\Users\alpha\Downloads\bd_7_10_2024_columns\COLUMN0801.jpg successfully.
  - Processing file: COLUMN0802.jpg
    - Processed C:\Users\alpha\Downloads\bd_7_10_2

In [29]:
# Create spatial-series plots (different variations)
create_spatial_series_plot_1(temperature_data)
create_spatial_series_plot_2(temperature_data)
create_spatial_series_plot_3(temperature_data)

In [30]:
print(temperature_data)

{1: {'TRACKS': [array([31.53285059, 31.57902982, 31.63672878, ..., 34.79086163,
       34.65590318, 33.94491777]), array([32.34989587, 32.30401277, 32.25811215, ..., 32.67058862,
       32.55615351, 32.55615351]), array([35.48579411, 35.8429411 , 35.89865219, ..., 31.45199396,
       31.54439708, 31.59057184])], 'PLATFORM': [array([32.33842674, 31.98233937, 31.78661589, ..., 32.03984432,
       32.06283859, 32.13179494]), array([32.39576146, 32.57904923, 33.32080558, ..., 31.30168615,
       31.27854505, 31.23224937]), array([], dtype=float64)], 'CEILING': [array([32.7963419 , 32.7506286 , 32.70489797, ..., 31.53285059,
       31.46354826, 31.61365254]), array([32.01684565, 31.98233937, 31.89027404, ..., 30.24417893,
       29.58867664, 30.27919258]), array([33.15003454, 33.59353984, 34.05804946, ..., 32.25811215,
       32.21219399, 32.23515527])], 'COLUMN': [array([32.59049546, 32.56760191, 32.62482761, ..., 31.30168615,
       31.29011616, 31.24382498]), array([32.72776545, 32.69346

       29.97539561, 30.01048903])]}}


In [31]:
# Loop over each station in temperature_data (stations 1 through 10)
for station in sorted(temperature_data.keys()):
    total_station_points = 0
    print(f"Station {station:02d}:")
    
    # Loop through each feature type for this station
    for feature in ['TRACKS', 'PLATFORM', 'CEILING', 'COLUMN']:
        # Each entry in temperature_data[station][feature] is a 1D numpy array.
        # We sum up the lengths of all these arrays to get the total number of data points.
        feature_points = sum(len(segment) for segment in temperature_data[station][feature])
        total_station_points += feature_points
        
        # Print the count for this feature
        print(f"  {feature}: {feature_points} temperature value{'s' if feature_points != 1 else ''}")
    
    # Print the total number of data points for the station
    print(f"  Total for station {station:02d}: {total_station_points} temperature value{'s' if total_station_points != 1 else ''}\n")

Station 01:
  TRACKS: 14971 temperature values
  PLATFORM: 5610 temperature values
  CEILING: 18809 temperature values
  COLUMN: 11032 temperature values
  Total for station 01: 50422 temperature values

Station 02:
  TRACKS: 6551 temperature values
  PLATFORM: 2015 temperature values
  CEILING: 13452 temperature values
  COLUMN: 4410 temperature values
  Total for station 02: 26428 temperature values

Station 03:
  TRACKS: 11315 temperature values
  PLATFORM: 6562 temperature values
  CEILING: 20484 temperature values
  COLUMN: 9719 temperature values
  Total for station 03: 48080 temperature values

Station 04:
  TRACKS: 6886 temperature values
  PLATFORM: 3847 temperature values
  CEILING: 10923 temperature values
  COLUMN: 7714 temperature values
  Total for station 04: 29370 temperature values

Station 05:
  TRACKS: 4088 temperature values
  PLATFORM: 31716 temperature values
  CEILING: 12666 temperature values
  COLUMN: 5228 temperature values
  Total for station 05: 53698 temper

In [32]:
records = []
for location, features in temperature_data.items():
        for feature, temp_list in features.items():
            for temp in temp_list:
                # Skip if there is no data in this segment.
                if temp.size == 0:
                    continue
                mean_temp = np.mean(temp)
                std_temp = np.std(temp)
                records.append({
                    'Station': location,
                    'Feature': feature,
                    'MeanTemp': mean_temp,
                    'StdTemp': std_temp
                })
    
if len(records) == 0:
    print("No temperature data available to cluster.")
    
# Convert records into a DataFrame.
df = pd.DataFrame(records)
    
# The scope of these changes made to
# pandas settings are local to with statement.
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df)

    Station   Feature  MeanTemp  StdTemp
0         1    TRACKS    32.094    1.594
1         1    TRACKS    31.617    1.634
2         1    TRACKS    33.145    2.409
3         1  PLATFORM    31.563    0.592
4         1  PLATFORM    31.964    0.702
5         1   CEILING    32.750    0.662
6         1   CEILING    32.899    1.082
7         1   CEILING    35.221    1.936
8         1    COLUMN    32.282    0.905
9         1    COLUMN    32.557    0.649
10        1    COLUMN    33.029    0.859
11        2    TRACKS    32.414    1.854
12        2    TRACKS    30.418    1.590
13        2  PLATFORM    30.953    0.953
14        2   CEILING    32.754    1.390
15        2   CEILING    30.782    1.149
16        2    COLUMN    30.946    0.524
17        2    COLUMN    30.502    0.588
18        3    TRACKS    29.651    0.946
19        3    TRACKS    26.945    1.095
20        3    TRACKS    32.440    1.635
21        3  PLATFORM    28.229    0.557
22        3  PLATFORM    32.440    1.628
23        3   CE

In [33]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from flirimageextractor import FlirImageExtractor

In [34]:
# --------------------------
# Configuration / Inputs
# --------------------------

THERMAL_DIR = r"C:\Users\alpha\OneDrive\Documents\FLIR\FLIR Thermal Studio Import\DCIM\100_FLIR\ALL Subway Lines - NYC"
MASK_DIRS = {
    'TRACKS': r"C:\Users\alpha\Downloads\ALL Subway Lines Tracks - NYC Resized",
    'PLATFORM': r"C:\Users\alpha\Downloads\ALL Subway Lines Platform - NYC Resized",
    'CEILING': r"C:\Users\alpha\Downloads\ALL Subway Lines Ceilings - NYC Resized",
    'COLUMNS': r"C:\Users\alpha\Downloads\ALL Subway Lines Columns - NYC Resized"
}
EXIFTOOL_PATH = r"C:\Users\alpha\exiftool-12.92_64\exiftool.exe"
METADATA_CSV = r"C:\Users\alpha\Downloads\station_metadata.csv"

In [35]:
# --------------------------
# Load Metadata & Build Station Pattern
# --------------------------
print("[START] Loading station metadata for regex...")
meta_df = pd.read_csv(METADATA_CSV)
station_abbrs = sorted(meta_df['abbr'].astype(str).unique(), key=lambda x: -len(x))
station_pattern = "|".join(re.escape(s) for s in station_abbrs)
print(f"[INFO] Station abbreviations: {station_abbrs}")

[START] Loading station metadata for regex...
[INFO] Station abbreviations: ['4A9S', 'WTC', '161', '167', '170', '174', '182', 'BPB', '205', 'JFK', 'BRA', 'SF', 'WS', 'BG', '14', '42', '34', '59', '63', '72', '86', '96', 'TA', 'FR', 'KR', '23', 'QP', 'JH', '71', 'KG', 'VW', 'JC', 'AA', 'US', 'PA', '25', '36', '45', '53', '77', '95']


In [36]:
def process_directories_with_logging(thermal_dir, mask_dirs, exiftool_path):
    """
    Reads FLIR images and corresponding masks, extracts temperatures,
    and logs progress for each image with try/atch for errors.
    Returns:
      1) data_dict: {(line, station): {feature: [temps]}}
      2) full_df: DataFrame with ['Station','Date','Feature','Temperature']
    """
    import os
    import re
    import numpy as np
    import pandas as pd
    from flirimageextractor import FlirImageExtractor
    from PIL import Image

    # Build filename regex (same as before)
    regex = (
        r"^FLIR(\d{2})(\d{2})(\d{2})"      # MM DD YY
        r"([A-Z0-9]+?)"                    # non-greedy line code
        r"(" + station_pattern + r")"      # station abbreviation
        r"([MAE])(\d{2})\.jpg$"            # time-of-day + index
    )
    pattern = re.compile(regex)

    data_dict = {}
    time_rows = []

    all_files = sorted(os.listdir(thermal_dir))
    print(f"[INFO] Found {len(all_files)} files in '{thermal_dir}'")

    for idx, fname in enumerate(all_files, start=1):
        print(f"[PROCESS] ({idx}/{len(all_files)}) {fname}")
        match = pattern.match(fname)
        if not match:
            print("  [SKIP] filename does not match expected pattern.")
            continue

        month, day, year, line_code, station, tod, num = match.groups()
        date_str = f"20{year}-{month}-{day}"
        key = (line_code, station)
        data_dict.setdefault(key, {feat: [] for feat in mask_dirs})

        thermal_path = os.path.join(thermal_dir, fname)
        if not os.path.isfile(thermal_path):
            print(f"  [WARN] Thermal file missing: {thermal_path}")
            continue

        # Attempt to load FLIR image
        try:
            flir = FlirImageExtractor(exiftool_path)
            flir.process_image(thermal_path, RGB=True)
            print("  [OK] Loaded thermal image.")
        except Exception as e:
            print(f"  [ERROR] Could not process thermal image: {e}")
            continue

        # For each feature, attempt to load mask and extract temperatures
        for feature, mask_dir in mask_dirs.items():
            mask_fname = f"{feature}{month}{day}{year}{line_code}{station}{tod}{num}.jpg"
            mask_path = os.path.join(mask_dir, mask_fname)
            print(f"    -> {feature}: checking mask '{mask_fname}'")

            if not os.path.isfile(mask_path):
                print(f"      [MISSING] {mask_path}")
                continue

            # Load mask as binary array
            try:
                img = Image.open(mask_path).convert("L")
                mask_arr = np.array(img) > 0
                print("      [OK] Loaded mask.")
            except Exception as e:
                print(f"      [ERROR] Could not load mask image: {e}")
                continue

            # Extract temperatures for this mask
            try:
                temps_arr = extract_temp_for_class(flir, mask_arr, plot=0).compressed()
                count = len(temps_arr)
                print(f"      [OK] Extracted {count} temperature values.")
            except Exception as e:
                print(f"      [ERROR] Temperature extraction failed: {e}")
                continue

            # Store pixel values in data_dict
            data_dict[key][feature].extend(temps_arr.tolist())

            # Also append to time_rows with station and date
            for t in temps_arr:
                time_rows.append({
                    'Station': station,
                    'Date': date_str,
                    'Feature': feature,
                    'Temperature': float(t)
                })

    print("[INFO] Processing complete.")
    full_df = pd.DataFrame(time_rows)
    return data_dict, full_df

In [37]:
# --------------------------
# Compute Daily Means per Station
# --------------------------

def compute_daily_means(full_df):
    """
    Given full_df (one row per pixel: Station, Date, Feature, Temperature),
    return a new DataFrame daily_means_df with columns:
       ['Station','Date','Feature','MeanTemperature']

    i.e. group by (Station, Date, Feature), compute .mean().
    """
    grouped = full_df.groupby(['Station','Date','Feature'])['Temperature']
    daily_means = grouped.mean().reset_index(name='MeanTemperature')
    return daily_means

In [38]:
# --------------------------
# Data Flattening
# --------------------------
def flatten_data(data):
    rows = []
    for (line_code, station), feats in data.items():
        for feature, temps in feats.items():
            rows.extend([{'Line': line_code, 'Station': station, 'Feature': feature, 'Temperature': t}
                         for t in temps])
    return pd.DataFrame(rows)

In [42]:
def box_plot_feature_dailymeans(daily_means_df, metadata_df, feature):
    """
    Box‐and‐whisker of station‐daily means, coloring all boxes according
    to the feature, clamping y between 20 and 45 °C, and drawing exactly
    four vertical lines at the borough boundaries:
      • x = –0.5  (left edge)
      • immediately after station '96'  (Manhattan→Bronx)
      • immediately after station '205' (Bronx→Queens)
      • immediately after station 'JC'  (Queens→Brooklyn)
    This version de‐duplicates metadata so that '42' only appears once.
    """
    # 1) Color mapping (handle both "COLUMN" and "COLUMNS")
    color_map = {
        'PLATFORM': 'red',
        'COLUMN':   'yellow',
        'COLUMNS':  'yellow',
        'CEILING':  'green',
        'TRACKS':   'purple'
    }
    feat = feature.upper()

    # 2) If DataFrame has 'COLUMNS' instead of 'COLUMN', switch automatically
    if feat not in daily_means_df['Feature'].unique():
        if feat == 'COLUMN' and 'COLUMNS' in daily_means_df['Feature'].unique():
            feat = 'COLUMNS'
        else:
            print(f"[WARN] No data for feature '{feature}' (checked '{feat}').")
            return

    box_color = color_map[feat]

    # 3) Filter to only that feature
    df_feat = daily_means_df[daily_means_df['Feature'] == feat]
    if df_feat.empty:
        print(f"[WARN] No rows found for feature '{feat}'.")
        return

    # 4) De‐duplicate metadata by 'abbr', keeping first occurrence
    meta_unique = metadata_df.drop_duplicates(subset='abbr', keep='first')
    all_abbrs = meta_unique['abbr'].tolist()

    # 5) Build station_order in the metadata sequence, but only those actually present
    station_order = [s for s in all_abbrs if s in df_feat['Station'].unique()]

    # 6) Compute borough‐boundary indices using the unique metadata rows
    boroughs = meta_unique.set_index('abbr').loc[station_order, 'borough']
    boundaries = [-0.5]
    if '96' in station_order:
        boundaries.append(station_order.index('96') + 0.5)
    if '205' in station_order:
        boundaries.append(station_order.index('205') + 0.5)
    if 'JC' in station_order:
        boundaries.append(station_order.index('JC') + 0.5)

    # 7) Fix y‐axis range to [20, 45]
    y_min, y_max = 25, 45

    plt.figure(figsize=(18, 6))
    ax = sns.boxplot(
        x='Station',
        y='MeanTemperature',
        data=df_feat,
        order=station_order,
        color=box_color,
        showfliers=False,
        whis=1.5
    )

    # 8) Draw the four vertical separators
    for xline in boundaries:
        ax.axvline(xline, color='gray', linestyle='--')

    # 9) Clamp x‐limits so there’s no extra blank at the right
    ax.set_xlim(-0.5, len(station_order) - 0.5)
    ax.set_ylim(y_min, y_max)

    ax.set_xticklabels(station_order, rotation=90, fontsize=8, fontweight='bold')
    ax.set_title(f"{feat.title()} (station‐daily means)", fontsize=14, fontweight='bold')
    ax.set_xlabel("Station", fontsize=12, fontweight='bold')
    ax.set_ylabel("Mean Temperature (°C)", fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.show()

In [43]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def time_series_boxplot_dailymeans(daily_means_df, feature):
    """
    Time‐series boxplot of station‐daily means for a given feature,
    clamped to [20, 45] °C, with feature‐specific color. 
    Skips the first two dates, and formats x‐ticks as 'Mon, MM/DD/YY'.
    """
    color_map = {
        'PLATFORM': 'red',
        'COLUMN':   'yellow',
        'COLUMNS':  'yellow',
        'CEILING':  'green',
        'TRACKS':   'purple'
    }
    feat = feature.upper()

    # If the DataFrame uses "COLUMNS" instead of "COLUMN", switch
    if feat not in daily_means_df['Feature'].unique():
        if feat == 'COLUMN' and 'COLUMNS' in daily_means_df['Feature'].unique():
            feat = 'COLUMNS'
        else:
            print(f"[WARN] No data for feature '{feature}' (checked '{feat}').")
            return

    box_color = color_map[feat]
    df_feat = daily_means_df[daily_means_df['Feature'] == feat].copy()
    if df_feat.empty:
        print(f"[WARN] No rows found for feature '{feat}'.")
        return

    df_feat['Date'] = pd.to_datetime(df_feat['Date'])
    dates_present = sorted(df_feat['Date'].unique())

    # Skip the first two dates (they were from years 2002 & 2023 and contain no data)
    if len(dates_present) > 2:
        dates_present = dates_present[2:]
    else:
        print("[WARN] Less than three unique dates; nothing to plot after skipping first two.")
        return

    # Filter df_feat to only these dates
    df_feat = df_feat[df_feat['Date'].isin(dates_present)]

    # Clamp y‐axis to [20, 45]
    y_min, y_max = 20, 45

    plt.figure(figsize=(14, 6))
    ax = sns.boxplot(
        x='Date',
        y='MeanTemperature',
        data=df_feat,
        order=dates_present,
        color=box_color,
        showfliers=False,
        whis=1.5
    )
    ax.set_ylim(y_min, y_max)

    # Format x‐tick labels as 'Mon, MM/DD/YY'
    formatted_labels = [d.strftime('%a, %m/%d/%y') for d in dates_present]
    ax.set_xticks(range(len(dates_present)))
    ax.set_xticklabels(formatted_labels, rotation=45, fontsize=8)

    ax.set_title(f"{feat.title()} (daily distribution of station‐means)", fontsize=14, fontweight='bold')
    ax.set_xlabel("Date", fontsize=12, fontweight='bold')
    ax.set_ylabel("Mean Temperature (°C)", fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.show()

In [59]:
# 1) Process everything and build full_df (pixel‐level with station+date)
data_dict, full_df = process_directories_with_logging(THERMAL_DIR, MASK_DIRS, EXIFTOOL_PATH)

[INFO] Found 2277 files in 'C:\Users\alpha\OneDrive\Documents\FLIR\FLIR Thermal Studio Import\DCIM\100_FLIR\ALL Subway Lines - NYC'
[PROCESS] (1/2277) FLIR070224BD161A01.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS070224BD161A01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 2940 temperature values.
    -> PLATFORM: checking mask 'PLATFORM070224BD161A01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 18281 temperature values.
    -> CEILING: checking mask 'CEILING070224BD161A01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 8759 temperature values.
    -> COLUMNS: checking mask 'COLUMNS070224BD161A01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 0 temperature values.
[PROCESS] (2/2277) FLIR070224BD161A02.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS070224BD161A02.jpg'
      [OK] Loaded mask.
      [OK] Extracted 2293 temperature values.
    -> PLATFORM: checking mask 'PLATFORM070224BD161A02.jpg'
      [OK] Loaded mask.
      

    -> COLUMNS: checking mask 'COLUMNS070224EFMRJCA01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 3704 temperature values.
[PROCESS] (15/2277) FLIR070224EFMRJCA02.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS070224EFMRJCA02.jpg'
      [OK] Loaded mask.
      [OK] Extracted 5037 temperature values.
    -> PLATFORM: checking mask 'PLATFORM070224EFMRJCA02.jpg'
      [OK] Loaded mask.
      [OK] Extracted 3961 temperature values.
    -> CEILING: checking mask 'CEILING070224EFMRJCA02.jpg'
      [OK] Loaded mask.
      [OK] Extracted 7673 temperature values.
    -> COLUMNS: checking mask 'COLUMNS070224EFMRJCA02.jpg'
      [OK] Loaded mask.
      [OK] Extracted 0 temperature values.
[PROCESS] (16/2277) FLIR070224EFMRJCA03.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS070224EFMRJCA03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 1680 temperature values.
    -> PLATFORM: checking mask 'PLATFORM070224EFMRJCA03.jpg'
      [OK] Loaded mask.


      [OK] Loaded mask.
      [OK] Extracted 14837 temperature values.
[PROCESS] (29/2277) FLIR070224RPAA01.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS070224RPAA01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 13772 temperature values.
    -> PLATFORM: checking mask 'PLATFORM070224RPAA01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 13787 temperature values.
    -> CEILING: checking mask 'CEILING070224RPAA01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 10523 temperature values.
    -> COLUMNS: checking mask 'COLUMNS070224RPAA01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 13721 temperature values.
[PROCESS] (30/2277) FLIR070224RUSA01.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS070224RUSA01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 14980 temperature values.
    -> PLATFORM: checking mask 'PLATFORM070224RUSA01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 14980 temperature values.
    -> CEILING: checking ma

      [OK] Extracted 3356 temperature values.
[PROCESS] (43/2277) FLIR070324BD167A01.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS070324BD167A01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 2176 temperature values.
    -> PLATFORM: checking mask 'PLATFORM070324BD167A01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 2042 temperature values.
    -> CEILING: checking mask 'CEILING070324BD167A01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 5533 temperature values.
    -> COLUMNS: checking mask 'COLUMNS070324BD167A01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 4154 temperature values.
[PROCESS] (44/2277) FLIR070324BD167A02.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS070324BD167A02.jpg'
      [OK] Loaded mask.
      [OK] Extracted 1922 temperature values.
    -> PLATFORM: checking mask 'PLATFORM070324BD167A02.jpg'
      [OK] Loaded mask.
      [OK] Extracted 16505 temperature values.
    -> CEILING: checking mask 'CEILING070

      [OK] Extracted 4841 temperature values.
[PROCESS] (57/2277) FLIR070324BD205A03.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS070324BD205A03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 13739 temperature values.
    -> PLATFORM: checking mask 'PLATFORM070324BD205A03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 0 temperature values.
    -> CEILING: checking mask 'CEILING070324BD205A03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 3718 temperature values.
    -> COLUMNS: checking mask 'COLUMNS070324BD205A03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 1375 temperature values.
[PROCESS] (58/2277) FLIR070324BD205A04.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS070324BD205A04.jpg'
      [OK] Loaded mask.
      [OK] Extracted 6566 temperature values.
    -> PLATFORM: checking mask 'PLATFORM070324BD205A04.jpg'
      [OK] Loaded mask.
      [OK] Extracted 886 temperature values.
    -> CEILING: checking mask 'CEILING070324B

...

    -> PLATFORM: checking mask 'PLATFORM073124NQ86M02.jpg'
      [OK] Loaded mask.
      [OK] Extracted 4910 temperature values.
    -> CEILING: checking mask 'CEILING073124NQ86M02.jpg'
      [OK] Loaded mask.
      [OK] Extracted 0 temperature values.
    -> COLUMNS: checking mask 'COLUMNS073124NQ86M02.jpg'
      [OK] Loaded mask.
      [OK] Extracted 0 temperature values.
[PROCESS] (2215/2277) FLIR073124NQ86M03.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS073124NQ86M03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 1098 temperature values.
    -> PLATFORM: checking mask 'PLATFORM073124NQ86M03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 4085 temperature values.
    -> CEILING: checking mask 'CEILING073124NQ86M03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 0 temperature values.
    -> COLUMNS: checking mask 'COLUMNS073124NQ86M03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 2871 temperature values.
[PROCESS] (2216/2277) FLIR073124NQ96M01.jpg
 

      [OK] Extracted 11568 temperature values.
    -> CEILING: checking mask 'CEILING0801241WTCA03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 5926 temperature values.
    -> COLUMNS: checking mask 'COLUMNS0801241WTCA03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 5818 temperature values.
[PROCESS] (2229/2277) FLIR08012423WSA01.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS08012423WSA01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 10995 temperature values.
    -> PLATFORM: checking mask 'PLATFORM08012423WSA01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 0 temperature values.
    -> CEILING: checking mask 'CEILING08012423WSA01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 6178 temperature values.
    -> COLUMNS: checking mask 'COLUMNS08012423WSA01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 3066 temperature values.
[PROCESS] (2230/2277) FLIR08012423WSA02.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS08012423WSA

      [OK] Extracted 15237 temperature values.
    -> CEILING: checking mask 'CEILING080124734A02.jpg'
      [OK] Loaded mask.
      [OK] Extracted 2775 temperature values.
    -> COLUMNS: checking mask 'COLUMNS080124734A02.jpg'
      [OK] Loaded mask.
      [OK] Extracted 3899 temperature values.
[PROCESS] (2243/2277) FLIR080124734A03.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS080124734A03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 0 temperature values.
    -> PLATFORM: checking mask 'PLATFORM080124734A03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 3718 temperature values.
    -> CEILING: checking mask 'CEILING080124734A03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 5838 temperature values.
    -> COLUMNS: checking mask 'COLUMNS080124734A03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 0 temperature values.
[PROCESS] (2244/2277) FLIR080124742A01.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS080124742A01.jpg'
     

    -> CEILING: checking mask 'CEILING080124BDKRM03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 2915 temperature values.
    -> COLUMNS: checking mask 'COLUMNS080124BDKRM03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 3168 temperature values.
[PROCESS] (2257/2277) FLIR080124BDTAM01.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS080124BDTAM01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 18563 temperature values.
    -> PLATFORM: checking mask 'PLATFORM080124BDTAM01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 18563 temperature values.
    -> CEILING: checking mask 'CEILING080124BDTAM01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 12974 temperature values.
    -> COLUMNS: checking mask 'COLUMNS080124BDTAM01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 6332 temperature values.
[PROCESS] (2258/2277) FLIR080124BDTAM02.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS080124BDTAM02.jpg'
      [OK] Loaded mask.
      [OK]

      [OK] Loaded mask.
      [OK] Extracted 6509 temperature values.
    -> COLUMNS: checking mask 'COLUMNS080124FQ86M02.jpg'
      [OK] Loaded mask.
      [OK] Extracted 0 temperature values.
[PROCESS] (2271/2277) FLIR080124FQ86M03.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS080124FQ86M03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 1185 temperature values.
    -> PLATFORM: checking mask 'PLATFORM080124FQ86M03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 1113 temperature values.
    -> CEILING: checking mask 'CEILING080124FQ86M03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 17814 temperature values.
    -> COLUMNS: checking mask 'COLUMNS080124FQ86M03.jpg'
      [OK] Loaded mask.
      [OK] Extracted 0 temperature values.
[PROCESS] (2272/2277) FLIR080124FQ96M01.jpg
  [OK] Loaded thermal image.
    -> TRACKS: checking mask 'TRACKS080124FQ96M01.jpg'
      [OK] Loaded mask.
      [OK] Extracted 10797 temperature values.
    -> PLATFORM: checking ma

In [60]:
# 2) Compute daily‐means:
daily_means_df = compute_daily_means(full_df)

In [61]:
print(f"[INFO] Total station‐daily mean samples: {len(daily_means_df)}")

[INFO] Total station‐daily mean samples: 1906


In [62]:
# Replace with your actual thermal directory
THERMAL_DIR = r"C:\Users\alpha\OneDrive\Documents\FLIR\FLIR Thermal Studio Import\DCIM\100_FLIR\ALL Subway Lines - NYC"

print("[INFO] Scanning for files with year ≠ 24 (i.e., 2002 or 2023)…\n")

for fname in sorted(os.listdir(THERMAL_DIR)):
    if not fname.upper().startswith("FLIR") or not fname.upper().endswith(".JPG"):
        continue

    # 'FLIR' + MM(4:6) + DD(6:8) + YY(8:10) + ...
    if len(fname) < 10:
        continue

    yy = fname[8:10]
    try:
        year_int = int(yy)
    except ValueError:
        continue

    # We only expect '24' for 2024. Anything else is out of bounds.
    if yy != "24":
        # Reconstruct full 4-digit year
        full_year = 2000 + year_int
        print(f"{fname}  → parsed year = {full_year}")

print("\n[INFO] Scan complete.")

[INFO] Scanning for files with year ≠ 24 (i.e., 2002 or 2023)…


[INFO] Scan complete.


In [63]:
# 3) Location‐series boxplot (using daily means across stations):
box_plot_feature_dailymeans(daily_means_df, meta_df, 'PLATFORM')
box_plot_feature_dailymeans(daily_means_df, meta_df, 'TRACKS')
box_plot_feature_dailymeans(daily_means_df, meta_df, 'CEILING')
box_plot_feature_dailymeans(daily_means_df, meta_df, 'COLUMN')

In [64]:
# 4) Time‐series boxplot (using daily means across stations):
time_series_boxplot_dailymeans(daily_means_df, 'PLATFORM')
time_series_boxplot_dailymeans(daily_means_df, 'TRACKS')
time_series_boxplot_dailymeans(daily_means_df, 'CEILING')
time_series_boxplot_dailymeans(daily_means_df, 'COLUMN')

In [136]:
# --------------------------
# Air Temperature Processing
# --------------------------
def process_airtemp(csv_path):
    """
    Load airtemp_value.csv and return DataFrame with:
      ['Date', 'abbrev', 'streetlevel_airtemp_c', 'turnstile_level_airtemp_c']
    Date parsed as datetime.date
    """
    df = pd.read_csv(csv_path, parse_dates=['date'])
    df.rename(columns={'date':'Date','abbrev':'Station'}, inplace=True)
    # Keep only necessary columns
    return df[['Date','Station','streetlevel_airtemp_c','turnstile_level_airtemp_c']]

In [137]:
# --------------------------
# Boxplot + AirTemp Overlay
# --------------------------
def box_plot_with_airtemp(daily_means_df, air_df, meta_df, feature):
    """
    Draw platform/columns boxplot + overlay air temperatures on the same axes.
    Stations ordered by meta_df['abbr'], capped y‐axis, borough separators.
    """
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns

    # Determine feature naming
    feat = feature.upper()
    if feat not in daily_means_df['Feature'].unique():
        if feat == 'COLUMN' and 'COLUMNS' in daily_means_df['Feature'].unique():
            feat = 'COLUMNS'
        else:
            print(f"[WARN] No thermal data for feature '{feature}'")
            return

    # Filter daily means for this feature
    df_feat = daily_means_df[daily_means_df['Feature'] == feat]
    if df_feat.empty:
        print(f"[WARN] Empty daily_means for '{feat}'")
        return

    # Station order
    meta_unique = meta_df.drop_duplicates(subset='abbr', keep='first')
    stations = [s for s in meta_unique['abbr'] if s in df_feat['Station'].unique()]

    # Build boxplot
    fig, ax = plt.subplots(figsize=(18,6))
    color_map = {'PLATFORM':'red','COLUMNS':'yellow','CEILING':'green','TRACKS':'purple'}
    sns.boxplot(
        x='Station', y='MeanTemperature', data=df_feat,
        order=stations, color=color_map.get(feat,'gray'),
        showfliers=False, whis=1.5, ax=ax
    )

    # Borough separators
    boundaries = [-0.5]
    for code in ['96','205','JC']:
        if code in stations:
            boundaries.append(stations.index(code) + 0.5)
    for x in boundaries:
        ax.axvline(x, color='gray', linestyle='--')

    # Process air_df: unify column names
    air = air_df.copy()
    if 'date' in air.columns:
        air['Date'] = pd.to_datetime(air['date'], errors='coerce')
    if 'Date' not in air.columns:
        air['Date'] = pd.to_datetime(air['Date'], errors='coerce')
    # identify air columns
    if 'streetlevel_airtemp_c' in air.columns and 'turnstile_level_airtemp_c' in air.columns:
        stcol, trcol = 'streetlevel_airtemp_c', 'turnstile_level_airtemp_c'
    else:
        stcol = trcol = None
        print("[WARN] No recognized air columns in air_df")

    # Compute mean air temps per station
    if stcol and trcol:
        grp = air.groupby('Station')[[stcol, trcol]].mean().reset_index()
        street_vals = [grp.set_index('Station').at[s, stcol] if s in grp['Station'].values else np.nan for s in stations]
        turn_vals   = [grp.set_index('Station').at[s, trcol] if s in grp['Station'].values else np.nan for s in stations]
        x = np.arange(len(stations))
        ax.plot(x, street_vals,
                linestyle='--', color='red', marker='o', markerfacecolor='black', label='Street Air Temp')
        ax.plot(x, turn_vals,
                linestyle='--', color='lightblue', marker='o', markerfacecolor='navy', label='Turnstile Air Temp')
        ax.legend(loc='lower right')

    # Final formatting
    ax.set_xlim(-0.5, len(stations) - 0.5)
    ax.set_ylim(25,45)
    ax.set_xticklabels(stations, rotation=90, fontsize=8, fontweight='bold')
    
    # Get all tick labels
    labels_y = ax.get_yticklabels()

    # Set fontweight to bold for each label
    for label in labels_y:
        label.set_fontweight('bold')
    
    ax.set_title(f"{feat.title()} (station‐daily means)", fontsize=14, fontweight='bold')
    ax.set_xlabel("Station", fontsize=12, fontweight='bold')
    ax.set_ylabel("Mean Temperature (°C)", fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.show()

In [160]:
# --------------------------
# Time-Series + AirTemp Overlay (single figure)
# --------------------------
def time_series_with_airtemp(daily_means_df, air_df, meta_df, feature):
    """
    Single-figure time-series boxplot + air-temp overlay.
    Uses integer x positions for perfect alignment.
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    import pandas as pd
    import numpy as np

    # Normalize feature
    feat = feature.upper()
    if feat not in daily_means_df['Feature'].unique():
        if feat=='COLUMN' and 'COLUMNS' in daily_means_df['Feature'].unique():
            feat='COLUMNS'
        else:
            print(f"[WARN] No data for feature '{feature}'")
            return

    # Filter for feature
    df_feat = daily_means_df[daily_means_df['Feature']==feat].copy()
    if df_feat.empty:
        print(f"[WARN] No rows for feature '{feat}'")
        return
    df_feat['Date'] = pd.to_datetime(df_feat['Date'])
    dates = sorted(df_feat['Date'].unique())
    # Map dates to integer positions
    date_to_x = {d:i for i,d in enumerate(dates)}
    df_feat['x'] = df_feat['Date'].map(date_to_x)

    # Air daily means
    air = air_df.copy()
    air['Date'] = pd.to_datetime(air['Date'], errors='coerce')
    air = air[air['Date'].isin(dates)]
    air_daily = air.groupby('Date').agg(
        street=('streetlevel_airtemp_c','mean'),
        turn   =('turnstile_level_airtemp_c','mean')
    ).reindex(dates).reset_index()
    air_daily['x'] = air_daily['Date'].map(date_to_x)

    # Plot
    fig, ax = plt.subplots(figsize=(14,6))
    # Boxplot at integer x
    sns.boxplot(
        x='x', y='MeanTemperature', data=df_feat,
        color={'PLATFORM':'red','COLUMNS':'yellow','CEILING':'green','TRACKS':'purple'}.get(feat,'gray'),
        showfliers=False, whis=1.5,
        ax=ax
    )
    # Overlay air temps
    ax.plot(air_daily['x'], air_daily['street'],
            linestyle='--', color='red', marker='o', markerfacecolor='black', label='Street Air Temp')
    ax.plot(air_daily['x'], air_daily['turn'],
            linestyle='--', color='lightblue', marker='o', markerfacecolor='navy', label='Turnstile Air Temp')

    # Formatting
    ax.set_ylim(25,45)
    ax.set_xlim(-0.5, len(dates)-0.5)
    ax.set_xticks(range(len(dates)))
    ax.set_xticklabels([d.strftime('%m/%d/%y') for d in dates], rotation=45, fontsize=8, fontweight='bold')
    
    # Get all tick labels
    labels_y = ax.get_yticklabels()

    # Set fontweight to bold for each label
    for label in labels_y:
        label.set_fontweight('bold')
    
    ax.set_title(f"{feat.title()} (daily distribution of station-means)", fontsize=14, fontweight='bold')
    ax.set_xlabel("Date", fontsize=12, fontweight='bold')
    ax.set_ylabel("Mean Temperature (°C)", fontsize=12, fontweight='bold')
    ax.legend(loc='upper left')
    plt.tight_layout()
    plt.show()

In [161]:
# 1) Process your air‐temp CSV (once):
air_df = process_airtemp(r"C:\Users\alpha\Downloads\airtemp_value.csv")

# 2) After you have your `daily_means_df` and `meta_df`, call:

# Location‐series + air:
box_plot_with_airtemp(daily_means_df, air_df, meta_df, 'PLATFORM')
box_plot_with_airtemp(daily_means_df, air_df, meta_df, 'COLUMNS')
box_plot_with_airtemp(daily_means_df, air_df, meta_df, 'CEILING')
box_plot_with_airtemp(daily_means_df, air_df, meta_df, 'TRACKS')

C:\NAK-TEMP\ipykernel_24724\2318160501.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(csv_path, parse_dates=['date'])


In [165]:
# Time‐series + air:
time_series_with_airtemp(daily_means_df, air_df, meta_df, 'PLATFORM')
time_series_with_airtemp(daily_means_df, air_df, meta_df, 'COLUMNS')
time_series_with_airtemp(daily_means_df, air_df, meta_df, 'CEILING')
time_series_with_airtemp(daily_means_df, air_df, meta_df, 'TRACKS')

In [52]:
# First: coerce Date → datetime and drop any 2002 or 2023 rows
df_all = daily_means_df.copy()
df_all['Date'] = pd.to_datetime(df_all['Date'])
df_all = df_all[~df_all['Date'].dt.year.isin([2002, 2023])]

In [53]:
# 1) TIME‐SERIES LINE PLOT PER FEATURE (Daily average across stations)
# ---------------------------------------------------------------------
# Filtered DataFrame is already in df_all.

# (a) Compute the daily average (across stations) for each feature
ts_df = (
    df_all
    .groupby(['Date','Feature'])['MeanTemperature']
    .mean()
    .reset_index()
)

# (b) Pivot so that each feature is a column
ts_pivot = ts_df.pivot(index='Date', columns='Feature', values='MeanTemperature')

# (c) Sort by date
ts_pivot = ts_pivot.sort_index()

# (d) Plot one line per feature
plt.figure(figsize=(14, 6))
feature_colors = {
    'PLATFORM':'red',
    'COLUMNS':'yellow',
    'CEILING':'green',
    'TRACKS':'purple'
}

for feat in ts_pivot.columns:
    # If the user originally named 'COLUMN' but pivot created 'COLUMNS', label it 'COLUMN'
    label = 'COLUMN' if feat == 'COLUMNS' else feat
    plt.plot(
        ts_pivot.index,
        ts_pivot[feat],
        label=label.title(),
        color=feature_colors.get(feat, 'gray'),
        linewidth=2
    )
plt.ylim(20, 45)
plt.title("Daily Mean Temperature (Averaged Across Stations)\nby Feature", fontsize=16)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Mean Temperature (°C)", fontsize=12)
plt.legend(title="Feature")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [54]:
# 2) HEATMAP: STATION × DATE GRID FOR PLATFORM DAILY MEANS (Blue→Red Scale)
# -------------------------------------------------------------------------
# We assume `df_all` (filtered to remove 2002/2023) and `meta_df` (with columns ['line','station_name','abbr',...]) already exist.

# (a) Filter to just PLATFORM
if 'TRACKS' in df_all['Feature'].unique():
    heat_df = df_all[df_all['Feature'] == 'TRACKS'].copy()
else:
    print("[WARN] 'TRACKS' not found in Feature column.")
    heat_df = pd.DataFrame(columns=['Station','Date','MeanTemperature'])

heat_df['Date'] = pd.to_datetime(heat_df['Date'])

# (b) Pivot into a matrix: index=Station, columns=Date, values=MeanTemperature
raw_heatmap = heat_df.pivot(index='Station', columns='Date', values='MeanTemperature')

# (c) Determine the “correct” station order from meta_df, dropping duplicates
meta_unique = meta_df.drop_duplicates(subset='abbr', keep='first')
full_order = list(meta_unique['abbr'])
# Keep only stations that actually appear in raw_heatmap.index
station_order = [s for s in full_order if s in raw_heatmap.index]

# (d) Reindex the pivot result so that its rows follow station_order
heatmap_data = raw_heatmap.reindex(index=station_order)

# (e) Plot the heatmap with a blue→red colormap (low=blue, high=red)
plt.figure(figsize=(16, 10))
sns.heatmap(
    heatmap_data,
    vmin=25, vmax=45,
    cmap='coolwarm',            # blue→red gradient
    cbar_kws={'label': 'Mean Temperature (°C)'},
    linewidths=0.5,
    linecolor='gray'
)
plt.title("Heatmap of Tracks Daily‐Mean Temperature\n(Stations × Dates)", fontsize=16)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Station", fontsize=12)

# (f) Format x‐ticks as “MM/DD/YY”
date_labels = [d.strftime('%m/%d/%y') for d in heatmap_data.columns]
plt.xticks(np.arange(len(date_labels)) + 0.5, date_labels, rotation=45, fontsize=8)

# (g) Use the station_order for the y‐axis ticks
plt.yticks(np.arange(len(station_order)) + 0.5, station_order, rotation=0, fontsize=8)

plt.tight_layout()
plt.show()

In [83]:
# HEATMAP: STATION × DATE GRID FOR COLUMNS DAILY MEANS (Blue→Red Scale)
# -------------------------------------------------------------------
# Using df_all (filtered for 2024+) and meta_df

# (a) Determine the correct feature key ('COLUMNS' or 'COLUMN')
if 'COLUMNS' in df_all['Feature'].unique():
    col_feature = 'COLUMNS'
elif 'COLUMN' in df_all['Feature'].unique():
    col_feature = 'COLUMN'
else:
    print("[WARN] Neither 'COLUMN' nor 'COLUMNS' found in Feature column.")
    col_feature = None

if col_feature:
    heat_df_cols = df_all[df_all['Feature'] == col_feature].copy()
    heat_df_cols['Date'] = pd.to_datetime(heat_df_cols['Date'])

    # (b) Pivot into matrix: index=Station, columns=Date, values=MeanTemperature
    raw_heatmap_cols = heat_df_cols.pivot(index='Station', columns='Date', values='MeanTemperature')

    # (c) Reproduce station order from meta_df (dropping duplicates)
    meta_unique = meta_df.drop_duplicates(subset='abbr', keep='first')
    full_order = list(meta_unique['abbr'])
    station_order = [s for s in full_order if s in raw_heatmap_cols.index]

    # (d) Reindex so rows follow station_order
    heatmap_data_cols = raw_heatmap_cols.reindex(index=station_order)

    # (e) Plot heatmap with blue→red colormap
    plt.figure(figsize=(16, 10))
    sns.heatmap(
        heatmap_data_cols,
        vmin=25, vmax=45,
        cmap='coolwarm',
        cbar_kws={'label': 'Mean Temperature (°C)'},
        linewidths=0.5,
        linecolor='gray'
    )
    plt.title("Heatmap of Column(s) Daily‐Mean Temperature\n(Stations × Dates)", fontsize=16)
    plt.xlabel("Date", fontsize=12)
    plt.ylabel("Station", fontsize=12)

    # (f) Format x‐ticks as “MM/DD/YY”
    date_labels_cols = [d.strftime('%m/%d/%y') for d in heatmap_data_cols.columns]
    plt.xticks(np.arange(len(date_labels_cols)) + 0.5, date_labels_cols, rotation=45, fontsize=8)

    # (g) Use station_order for y‐axis ticks
    plt.yticks(np.arange(len(station_order)) + 0.5, station_order, rotation=0, fontsize=8)

    plt.tight_layout()
    plt.show()


In [70]:
# 3) PAIRPLOT (CORRELATION) ACROSS FEATURES FOR EACH (Station, Date)
# -------------------------------------------------------------------
# Pivot daily_means_df to wide format: index = (Station, Date), columns = Feature
wide_df = (
    daily_means_df
    .pivot_table(
        index=['Station','Date'],
        columns='Feature',
        values='MeanTemperature'
    )
    .reset_index()
)

# Rename columns if necessary (e.g. “COLUMNS” → “COLUMN”)
wide_df = wide_df.rename(columns={'COLUMNS': 'COLUMN'})

# Drop rows that have any missing feature value
feature_cols = ['PLATFORM','COLUMN','CEILING','TRACKS']
wide_df_complete = wide_df.dropna(subset=feature_cols)

# Pairplot to see pairwise relationships
sns.pairplot(
    wide_df_complete[feature_cols],
    diag_kind='kde',
    plot_kws={'alpha': 0.6, 's': 40},
    corner=False
)
plt.suptitle("Pairwise Scatter + KDE of Station‐Daily Mean Temperatures\nAcross Features", y=1.02, fontsize=16)
plt.tight_layout()
plt.show()

C:\Users\alpha\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
C:\Users\alpha\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
C:\Users\alpha\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
C:\Users\alpha\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating

In [ ]:
import matplotlib.dates as mdates   # ← Add this line


# ──────────────────────────────────────────────────────────────────────────────
# ASSUMPTIONS:
#   1) `daily_means_df` already exists with columns:
#        ['Station', 'Date', 'Feature', 'MeanTemperature']
#   2) `meta_df` already exists with columns:
#        ['line', 'station_name', 'abbr', ...]
#   3) You have already filtered out any 2002/2023 data if present.
# ──────────────────────────────────────────────────────────────────────────────

# 1) COPY & FILTER FOR 2024 ONLY
df_all = daily_means_df.copy()
df_all['Date'] = pd.to_datetime(df_all['Date'])
df_all = df_all[df_all['Date'].dt.year == 2024]

# 2) DETERMINE FEATURE SPELLING FOR COLUMNS
df_all['Feature'] = df_all['Feature'].str.upper()
if 'COLUMNS' in df_all['Feature'].unique():
    col_key = 'COLUMNS'
elif 'COLUMN' in df_all['Feature'].unique():
    col_key = 'COLUMN'
else:
    raise RuntimeError("Neither 'COLUMN' nor 'COLUMNS' found in Feature column.")

# 3) REPRODUCE STATION ORDER FROM meta_df (deduplicated)
meta_unique = meta_df.drop_duplicates(subset='abbr', keep='first')
full_order = list(meta_unique['abbr'])
# Only keep those stations that actually appear in df_all
station_order = [s for s in full_order if s in df_all['Station'].unique()]

# 4) ASSIGN EACH STATION A Y‐POSITION (0,1,2,…) FOR PLOTTING
station_to_y = {s: i for i, s in enumerate(station_order)}

# 5) DEFINE FEATURE‐SPECIFIC X‐OFFSETS (fractions of a day)
offset_map = {
    'PLATFORM': -0.3,
    'COLUMN':   -0.1,
    'COLUMNS':  -0.1,
    'CEILING':  +0.1,
    'TRACKS':   +0.3
}

# 6) BUILD OUR “MEGA” SCATTER DATA
plot_rows = []
for _, row in df_all.iterrows():
    feat = row['Feature']
    date = row['Date']
    st   = row['Station']
    temp = row['MeanTemperature']
    date_num = mdates.date2num(date)     # (<-- mdates now defined)
    x_val = date_num + offset_map.get(feat, 0.0)
    y_val = station_to_y[st]
    plot_rows.append({
        'x': x_val,
        'y': y_val,
        'Feature': feat,
        'Temp': temp
    })

plot_df = pd.DataFrame(plot_rows)

# 7) SET UP FIGURE & AXIS
plt.figure(figsize=(16, len(station_order)*0.25 + 2))
ax = plt.gca()

# 8) DRAW SCATTER POINTS FOR EACH FEATURE
norm = plt.Normalize(20, 45)
cmap = plt.get_cmap('coolwarm')

for feat, _ in offset_map.items():
    subset = plot_df[plot_df['Feature'] == feat]
    if subset.empty:
        continue
    sc = ax.scatter(
        subset['x'],
        subset['y'],
        s=30,
        c=subset['Temp'],
        cmap=cmap,
        norm=norm,
        edgecolor='none',
        label=('COLUMN' if feat in ['COLUMN','COLUMNS'] else feat.title())
    )

# 9) FORMAT Y‐AXIS: station names
ax.set_yticks(list(station_to_y.values()))
ax.set_yticklabels(station_order, fontsize=8)
ax.set_ylabel("Station (in metadata order)", fontsize=12)

# 10) FORMAT X‐AXIS: dates
unique_dates = sorted(df_all['Date'].dt.normalize().unique())
date_nums = mdates.date2num(unique_dates)

ax.set_xticks(date_nums)
ax.set_xticklabels(
    [d.strftime('%a, %m/%d/%y') for d in unique_dates],
    rotation=45, ha='right', fontsize=8
)
ax.set_xlabel("Date (2024)", fontsize=12)

# 11) FIX Y‐LIM
ax.set_ylim(-0.5, len(station_order) - 0.5)

# 12) ADD COLORBAR
cbar = plt.colorbar(sc, ax=ax, pad=0.01)
cbar.set_label("Mean Temperature (°C)", fontsize=12)

# 13) ADD TITLE & LEGEND
ax.set_title("Mega‐Scatter: Daily Mean Temperature by (Feature × Station) × Date", fontsize=16, pad=20)
ax.legend(
    title="Feature",
    loc="upper left",
    bbox_to_anchor=(1.02, 1),
    fontsize=9,
    title_fontsize=10
)

plt.tight_layout()
plt.show()

In [38]:
# --------------------------
# Boxplot by Station
# --------------------------
def box_plot_feature(df, meta_df, feature):
    df_feat = df[df['Feature'] == feature]
    stations_present = sorted(df_feat['Station'].unique(), key=lambda x: meta_df['abbr'].tolist().index(x))
    boroughs = meta_df.set_index('abbr').loc[stations_present, 'borough']
    change_ix = [i for i in range(1, len(boroughs)) if boroughs.iloc[i] != boroughs.iloc[i-1]]
    low, high = df_feat['Temperature'].quantile([0.01, 0.99])

    plt.figure(figsize=(18, 6))
    ax = sns.boxplot(
        x='Station', y='Temperature', data=df_feat,
        order=stations_present, showfliers=False, whis=1.5
    )
    for ix in change_ix:
        ax.axvline(ix-0.5, color='gray', linestyle='--')
    ax.set_ylim(low, high)
    plt.xticks(rotation=90)
    plt.title(feature.title())
    plt.tight_layout()
    plt.show()

In [39]:
# --------------------------
# Time-series Boxplot
# --------------------------
def time_series_boxplot(time_df, feature):
    df_feat = time_df[time_df['Feature'] == feature].copy()
    df_feat['Date'] = pd.to_datetime(df_feat['Date'])
    dates = sorted(df_feat['Date'].unique())
    low, high = df_feat['Temperature'].quantile([0.01, 0.99])

    plt.figure(figsize=(14, 6))
    ax = sns.boxplot(
        x='Date', y='Temperature', data=df_feat,
        order=dates, showfliers=False, whis=1.5
    )
    ax.set_ylim(low, high)
    plt.xticks(rotation=45)
    plt.title(f"{feature.title()} Daily")
    plt.tight_layout()
    plt.show()

In [73]:
# --------------------------
# Execution (Jupyter)
# --------------------------
print("Loading metadata...")
meta_df = pd.read_csv(METADATA_CSV)
print("Processing images...")
data_dict, time_df = process_all_images(THERMAL_DIR, MASK_DIRS, EXIFTOOL_PATH)
df = flatten_data(data_dict)

box_plot_feature(df, meta_df, 'PLATFORM')
box_plot_feature(df, meta_df, 'TRACKS')
box_plot_feature(df, meta_df, 'CEILING')
box_plot_feature(df, meta_df, 'COLUMN')
time_series_boxplot(time_df, 'PLATFORM')
time_series_boxplot(time_df, 'TRACKS')
time_series_boxplot(time_df, 'CEILING')
time_series_boxplot(time_df, 'COLUMN')

Loading metadata...
Processing images...
[INFO] Validating ExifTool path...
[INFO] Using filename regex: ^FLIR(\d{2})(\d{2})(\d{2})([A-Z0-9]+?)(4A9S|WTC|161|167|170|174|182|BPB|205|JFK|BRA|SF|WS|BG|14|42|34|59|63|72|86|96|TA|FR|KR|23|QP|JH|71|KG|VW|JC|AA|US|PA|25|36|45|53|77|95)([MAE])(\d{2})\.jpg$
[INFO] Found 2251 FLIR files in C:\Users\alpha\OneDrive\Documents\FLIR\FLIR Thermal Studio Import\DCIM\100_FLIR\ALL Subway Lines - NYC
[PROCESS] 1/2251: FLIR070224BD161A01.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 2940 temps for TRACKS.
    [OK] Extracted 18281 temps for PLATFORM.
    [OK] Extracted 8759 temps for CEILING.
    [OK] Extracted 0 temps for COLUMNS.
[PROCESS] 2/2251: FLIR070224BD161A02.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 2293 temps for TRACKS.
    [OK] Extracted 1470 temps for PLATFORM.
    [OK] Extracted 5061 temps for CEILING.
    [OK] Extracted 7594 temps for COLUMNS.
[PROCESS] 3/2251: FLIR070224BD161A03.jpg
  [OK] Loaded thermal image.
    [OK] Extr

    [OK] Extracted 17783 temps for PLATFORM.
    [OK] Extracted 4278 temps for CEILING.
    [OK] Extracted 4536 temps for COLUMNS.
[PROCESS] 67/2251: FLIR070324BDFRA02.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 3218 temps for TRACKS.
    [OK] Extracted 2452 temps for PLATFORM.
    [OK] Extracted 4820 temps for CEILING.
    [OK] Extracted 9753 temps for COLUMNS.
[PROCESS] 68/2251: FLIR070324BDFRA03.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 0 temps for TRACKS.
    [OK] Extracted 0 temps for PLATFORM.
    [OK] Extracted 7090 temps for CEILING.
    [OK] Extracted 6394 temps for COLUMNS.
[PROCESS] 69/2251: FLIR070324BDFRA04.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 17893 temps for TRACKS.
    [OK] Extracted 17893 temps for PLATFORM.
    [OK] Extracted 17889 temps for CEILING.
    [OK] Extracted 2540 temps for COLUMNS.
[PROCESS] 70/2251: FLIR070324BDKRA01.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 3758 temps for TRACKS.
    [OK] Extracted 4048 temps for 

[PROCESS] 134/2251: FLIR070324R45A02.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 8670 temps for TRACKS.
    [OK] Extracted 1016 temps for PLATFORM.
    [OK] Extracted 5725 temps for CEILING.
    [OK] Extracted 1620 temps for COLUMNS.
[PROCESS] 135/2251: FLIR070324R45M01.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 11370 temps for TRACKS.
    [OK] Extracted 2301 temps for PLATFORM.
    [OK] Extracted 6217 temps for CEILING.
    [OK] Extracted 2580 temps for COLUMNS.
[PROCESS] 136/2251: FLIR070324R4A9SA01.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 12575 temps for TRACKS.
    [OK] Extracted 3168 temps for PLATFORM.
    [OK] Extracted 4734 temps for CEILING.
    [OK] Extracted 12502 temps for COLUMNS.
[PROCESS] 137/2251: FLIR070324R4A9SA02.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 7906 temps for TRACKS.
    [OK] Extracted 3219 temps for PLATFORM.
    [OK] Extracted 6105 temps for CEILING.
    [OK] Extracted 7906 temps for COLUMNS.
[PROCESS] 138/2251: FLIR0

    [OK] Extracted 3849 temps for COLUMNS.
[PROCESS] 168/2251: FLIR0708241SFA05.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 15042 temps for TRACKS.
    [OK] Extracted 15042 temps for PLATFORM.
    [OK] Extracted 10760 temps for CEILING.
    [OK] Extracted 2316 temps for COLUMNS.
[PROCESS] 169/2251: FLIR0708241SFA06.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 0 temps for TRACKS.
    [OK] Extracted 13167 temps for PLATFORM.
    [OK] Extracted 10297 temps for CEILING.
    [OK] Extracted 2782 temps for COLUMNS.
[PROCESS] 170/2251: FLIR0708241WTCA01.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 10324 temps for TRACKS.
    [OK] Extracted 2985 temps for PLATFORM.
    [OK] Extracted 7784 temps for CEILING.
    [OK] Extracted 1533 temps for COLUMNS.
[PROCESS] 171/2251: FLIR0708241WTCA02.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 0 temps for TRACKS.
    [OK] Extracted 0 temps for PLATFORM.
    [OK] Extracted 8196 temps for CEILING.
    [OK] Extracted 11108 temps fo

    [OK] Extracted 9727 temps for TRACKS.
    [OK] Extracted 1061 temps for PLATFORM.
    [OK] Extracted 4923 temps for CEILING.
    [OK] Extracted 9723 temps for COLUMNS.
[PROCESS] 236/2251: FLIR070824BDFRA03.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 1972 temps for TRACKS.
    [OK] Extracted 0 temps for PLATFORM.
    [OK] Extracted 6706 temps for CEILING.
    [OK] Extracted 2913 temps for COLUMNS.
[PROCESS] 237/2251: FLIR070824BDKRA01.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 11015 temps for TRACKS.
    [OK] Extracted 0 temps for PLATFORM.
    [OK] Extracted 8459 temps for CEILING.
    [OK] Extracted 1338 temps for COLUMNS.
[PROCESS] 238/2251: FLIR070824BDKRA02.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 5208 temps for TRACKS.
    [OK] Extracted 0 temps for PLATFORM.
    [OK] Extracted 7892 temps for CEILING.
    [OK] Extracted 3827 temps for COLUMNS.
[PROCESS] 239/2251: FLIR070824BDKRA03.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 2136 temps for TR

...

    [OK] Extracted 4616 temps for CEILING.
    [OK] Extracted 0 temps for COLUMNS.
[PROCESS] 1989/2251: FLIR072924FQ72M01.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 8908 temps for TRACKS.
    [OK] Extracted 4900 temps for PLATFORM.
    [OK] Extracted 4647 temps for CEILING.
    [OK] Extracted 0 temps for COLUMNS.
[PROCESS] 1990/2251: FLIR072924FQ72M02.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 2670 temps for TRACKS.
    [OK] Extracted 2424 temps for PLATFORM.
    [OK] Extracted 6265 temps for CEILING.
    [OK] Extracted 0 temps for COLUMNS.
[PROCESS] 1991/2251: FLIR072924FQ72M03.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 1246 temps for TRACKS.
    [OK] Extracted 3856 temps for PLATFORM.
    [OK] Extracted 3874 temps for CEILING.
    [OK] Extracted 0 temps for COLUMNS.
[PROCESS] 1992/2251: FLIR072924FQ86M01.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 7525 temps for TRACKS.
    [OK] Extracted 3094 temps for PLATFORM.
    [OK] Extracted 5901 temps for CE

[PROCESS] 2056/2251: FLIR073024BD182M01.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 1633 temps for TRACKS.
    [OK] Extracted 16399 temps for PLATFORM.
    [OK] Extracted 5090 temps for CEILING.
    [OK] Extracted 11624 temps for COLUMNS.
[PROCESS] 2057/2251: FLIR073024BD182M02.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 16958 temps for TRACKS.
    [OK] Extracted 18101 temps for PLATFORM.
    [OK] Extracted 6323 temps for CEILING.
    [OK] Extracted 10982 temps for COLUMNS.
[PROCESS] 2058/2251: FLIR073024BD182M03.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 2632 temps for TRACKS.
    [OK] Extracted 3028 temps for PLATFORM.
    [OK] Extracted 5009 temps for CEILING.
    [OK] Extracted 3215 temps for COLUMNS.
[PROCESS] 2059/2251: FLIR073024BD182M04.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 8652 temps for TRACKS.
    [OK] Extracted 18999 temps for PLATFORM.
    [OK] Extracted 8025 temps for CEILING.
    [OK] Extracted 6575 temps for COLUMNS.
[PROCESS] 2060

    [OK] Extracted 16237 temps for PLATFORM.
    [OK] Extracted 0 temps for CEILING.
    [OK] Extracted 3593 temps for COLUMNS.
[PROCESS] 2124/2251: FLIR073024R53M01.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 11939 temps for TRACKS.
    [OK] Extracted 2124 temps for PLATFORM.
    [OK] Extracted 4953 temps for CEILING.
    [OK] Extracted 3842 temps for COLUMNS.
[PROCESS] 2125/2251: FLIR073024R53M02.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 5430 temps for TRACKS.
    [OK] Extracted 4699 temps for PLATFORM.
    [OK] Extracted 3337 temps for CEILING.
    [OK] Extracted 6045 temps for COLUMNS.
[PROCESS] 2126/2251: FLIR073024R53M03.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 6709 temps for TRACKS.
    [OK] Extracted 14552 temps for PLATFORM.
    [OK] Extracted 0 temps for CEILING.
    [OK] Extracted 1298 temps for COLUMNS.
[PROCESS] 2127/2251: FLIR073024R77A01.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 9517 temps for TRACKS.
    [OK] Extracted 6040 temps f

    [OK] Extracted 4800 temps for COLUMNS.
[PROCESS] 2191/2251: FLIR073124BD205M02.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 9991 temps for TRACKS.
    [OK] Extracted 0 temps for PLATFORM.
    [OK] Extracted 1832 temps for CEILING.
    [OK] Extracted 5470 temps for COLUMNS.
[PROCESS] 2192/2251: FLIR073124BD205M03.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 18656 temps for TRACKS.
    [OK] Extracted 18645 temps for PLATFORM.
    [OK] Extracted 8313 temps for CEILING.
    [OK] Extracted 2728 temps for COLUMNS.
[PROCESS] 2193/2251: FLIR073124BD205M04.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 12549 temps for TRACKS.
    [OK] Extracted 2533 temps for PLATFORM.
    [OK] Extracted 0 temps for CEILING.
    [OK] Extracted 3145 temps for COLUMNS.
[PROCESS] 2194/2251: FLIR073124BDBPBM01.jpg
  [OK] Loaded thermal image.
    [OK] Extracted 4191 temps for TRACKS.
    [OK] Extracted 0 temps for PLATFORM.
    [OK] Extracted 6813 temps for CEILING.
    [OK] Extracted 4155 te

ValueError: min() arg is an empty sequence

In [74]:
time_series_boxplot(time_df, 'PLATFORM')
time_series_boxplot(time_df, 'TRACKS')
time_series_boxplot(time_df, 'CEILING')
time_series_boxplot(time_df, 'COLUMN')

ValueError: min() arg is an empty sequence

In [ ]:
# --------------------------
# Process Specific South Ferry Images
# --------------------------

def process_specific_images(suffixes, thermal_dir, mask_dirs, exiftool_path):
    regex = (
        r"^FLIR(\d{2})(\d{2})(\d{2})([A-Z0-9]+?)"
        + f"({'|'.join(map(re.escape, suffixes))})"
        + r"\.jpg$"
    )
    pattern = re.compile(regex)
    data = {}
    rows = []

    for fname in sorted(os.listdir(thermal_dir)):
        m = pattern.match(fname)
        if not m:
            continue
        _, _, _, line_code, suffix = m.groups()
        thermal_path = os.path.join(thermal_dir, fname)
        flir = FlirImageExtractor(exiftool_path)
        try:
            flir.process_image(thermal_path, RGB=True)
        except:
            continue

        for feature, mask_dir in mask_dirs.items():
            mask_fname = f"{feature}{suffix}.jpg"
            mask_path = os.path.join(mask_dir, mask_fname)
            if not os.path.isfile(mask_path):
                continue
            mask = load_mask_image(mask_path)
            try:
                temps_arr = extract_temp_for_class(flir, mask, plot=0).compressed()
            except:
                continue
            data.setdefault(suffix, {})[feature] = temps_arr.tolist()
            rows += [{'Suffix': suffix, 'Feature': feature, 'Temperature': t} for t in temps_arr]

    df_spec = pd.DataFrame(rows)
    return data, df_spec

In [76]:
print(f"[INFO] Total temperature values processed: {len(df)}")
# Print total count of temperature values processed
print(f"[INFO] Total temperature values processed: {len(df)}")

# Additional summary statistics
print("[INFO] Overall temperature distribution:")
print(df['Temperature'].describe())

print("[INFO] Temperature statistics by feature:")
print(df.groupby('Feature')['Temperature'].describe())

unique_dates = time_df['Date'].nunique()
print(f"[INFO] Number of unique dates processed: {unique_dates}")
print("[INFO] Data points per date:")
print(time_df['Date'].value_counts().sort_index())

[INFO] Total temperature values processed: 54703146
[INFO] Total temperature values processed: 54703146
[INFO] Overall temperature distribution:
count    5.470315e+07
mean     3.258277e+01
std      3.739934e+00
min      6.195873e+00
25%      3.025988e+01
50%      3.245677e+01
75%      3.457796e+01
max      2.560829e+02
Name: Temperature, dtype: float64
[INFO] Temperature statistics by feature:
               count       mean       std        min        25%        50%  \
Feature                                                                      
CEILING   14563879.0  33.164130  4.064271   9.541851  30.559553  32.946079   
COLUMNS   12375350.0  32.408953  3.508412  10.763666  30.267768  32.418947   
PLATFORM  11220546.0  32.230958  3.721368   6.195873  29.970831  32.058514   
TRACKS    16543371.0  32.439607  3.561391  10.763666  30.211863  32.407297   

                75%         max  
Feature                          
CEILING   35.171560  154.541433  
COLUMNS   34.383265  195.923163 

In [32]:
# Use the mean and standard deviation as features for clustering.
X = df[['MeanTemp', 'StdTemp']].values

# Explicitly set n_init to suppress the FutureWarning.
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)

df['Cluster'] = kmeans.fit_predict(X)

# Create the scatter plot.
plt.figure(figsize=(10, 7))
scatter = sns.scatterplot(data=df, x='MeanTemp', y='StdTemp', hue='Cluster',
                          style='Feature', palette='deep', s=100)
plt.title("Clustering of Subway Segments Based on Temperature")
plt.xlabel("Mean Temperature (°C)")
plt.ylabel("Temperature Standard Deviation (°C)")
plt.legend(title='Cluster / Feature', bbox_to_anchor=(1.05, 1), loc=2)
plt.tight_layout()
plt.show()

AttributeError: 'NoneType' object has no attribute 'split'

In [33]:
def create_clustering_plot(temperature_data, n_clusters=4):
    """
    Aggregates the temperature data from each segmented thermal image (for features such as
    TRACKS, PLATFORM, CEILING, and COLUMN), computes summary statistics (mean and standard deviation),
    clusters the segments using K-Means, and creates a scatter plot with:
    
      - x-axis: Mean Temperature (°C)
      - y-axis: Temperature Standard Deviation (°C)
      
    Points are colored by the K-Means cluster (unsupervised result) and have different markers
    based on the actual surface type (feature) from the segmentation.
    
    INPUTS:
        temperature_data : dict
            A dictionary created by process_directories() with keys = station numbers (1..10) and values = 
            dictionaries with keys corresponding to features (TRACKS, PLATFORM, CEILING, COLUMN). Each value is 
            a list of 1D numpy arrays (the extracted temperature values from the masked image).
        n_clusters : int, optional
            The number of clusters for K-Means (default is 4).
            
    OUTPUT:
        Displays a matplotlib scatter plot.
    """
    # Aggregate data records from the temperature_data dictionary.
    records = []
    for location, features in temperature_data.items():
        for feature, temp_list in features.items():
            for temp in temp_list:
                # Skip if there is no data in this segment.
                if temp.size == 0:
                    continue
                mean_temp = np.mean(temp)
                std_temp = np.std(temp)
                records.append({
                    'Station': station,
                    'Feature': feature,
                    'MeanTemp': mean_temp,
                    'StdTemp': std_temp
                })
    
    if len(records) == 0:
        print("No temperature data available to cluster.")
        return
    
    # Convert records into a DataFrame.
    df = pd.DataFrame(records)
    
    # Use the mean and standard deviation as features for clustering.
    X = df[['MeanTemp', 'StdTemp']].values

    # Explicitly set n_init to suppress the FutureWarning.
    kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)

    df['Cluster'] = kmeans.fit_predict(X)

    # Create the scatter plot.
    plt.figure(figsize=(10, 7))
    scatter = sns.scatterplot(data=df, x='MeanTemp', y='StdTemp', hue='Cluster',
                              style='Feature', palette='deep', s=100)
    plt.title("Clustering of Subway Segments Based on Temperature")
    plt.xlabel("Mean Temperature (°C)")
    plt.ylabel("Temperature Standard Deviation (°C)")
    plt.legend(title='Cluster / Feature', bbox_to_anchor=(1.05, 1), loc=2)
    plt.tight_layout()
    plt.show()

In [34]:
# Example of how you might call this function after processing your directories:
# (Assuming temperature_data has been created by process_directories.)
create_clustering_plot(temperature_data)

AttributeError: 'NoneType' object has no attribute 'split'

In [35]:
from sklearn.decomposition import PCA

def create_pca_clustering_plot(temperature_data, n_clusters=4):
    """
    This function extracts multiple features from each subway segment, performs K-Means
    clustering in that higher-dimensional feature space, then uses PCA to reduce to
    two dimensions for visualization.

    INPUT:
        temperature_data : dict
            Dictionary with the structure:
                temperature_data[station_number][feature] = [list of 1D arrays]
            Each 1D array represents the temperature values of a particular segment.

        n_clusters : int
            Number of clusters to use for K-Means (default 4).

    OUTPUT:
        Displays a 2D scatter plot where each point is a segment, colored by cluster
        and marked by the actual feature type.
    """

    # 1) Gather data from temperature_data into a list of records.
    records = []
    for station, features_dict in temperature_data.items():
        for feature_name, segment_list in features_dict.items():
            for segment in segment_list:
                # Skip empty segments
                if segment.size == 0:
                    continue

                # Compute multiple statistical features:
                mean_temp = np.mean(segment)
                std_temp = np.std(segment)
                min_temp = np.min(segment)
                max_temp = np.max(segment)
                median_temp = np.median(segment)
                iqr_temp = np.percentile(segment, 75) - np.percentile(segment, 25)

                # Store them in a dict (one record per segment)
                records.append({
                    'Station': station,
                    'Feature': feature_name,
                    'MeanTemp': mean_temp,
                    'StdTemp': std_temp,
                    'MinTemp': min_temp,
                    'MaxTemp': max_temp,
                    'MedianTemp': median_temp,
                    'IQR': iqr_temp
                })

    # If no records, nothing to cluster
    if len(records) == 0:
        print("No temperature data available to cluster.")
        return

    # 2) Convert the list of records into a pandas DataFrame
    df = pd.DataFrame(records)

    # 3) Select the columns (features) to be used for clustering
    feature_cols = ['MeanTemp', 'StdTemp', 'MinTemp', 'MaxTemp', 'MedianTemp', 'IQR']
    X = df[feature_cols].values

    # 4) Perform K-Means clustering in the 6D space
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    df['Cluster'] = kmeans.fit_predict(X)

    # 5) Apply PCA to reduce from 6 dimensions to 2 for visualization
    pca = PCA(n_components=2, random_state=42)
    pca_result = pca.fit_transform(X)

    # 6) Add the principal components (PC1, PC2) to the DataFrame
    df['PC1'] = pca_result[:, 0]
    df['PC2'] = pca_result[:, 1]

    # 7) Create a scatter plot of PC1 vs. PC2
    plt.figure(figsize=(10, 7))
    sns.scatterplot(
        data=df,
        x='PC1',
        y='PC2',
        hue='Cluster',      # Color points by cluster
        style='Feature',    # Use different markers for the known feature type
        s=100,
        palette='deep'
    )
    plt.title("PCA Clustering of Subway Segments (6D → 2D)")
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.legend(title='Cluster / Feature', bbox_to_anchor=(1.05, 1), loc=2)
    plt.tight_layout()
    plt.show()

In [36]:
create_pca_clustering_plot(temperature_data)

AttributeError: 'NoneType' object has no attribute 'split'

In [37]:
def pairwise_feature_visualization(temperature_data):
    """
    Creates a multi-feature DataFrame from the temperature_data dictionary,
    then uses seaborn's pairplot to visualize pairwise relationships among
    mean, std, min, max, median, and IQR. Points are colored by the actual
    'Feature' (TRACKS, PLATFORM, CEILING, COLUMN).
    
    Parameters
    ----------
    temperature_data : dict
        Nested dictionary structure:
            temperature_data[station][feature] = list of 1D numpy arrays
        Each 1D array contains pixel-level temperature values for that segment.

    Returns
    -------
    None (displays a PairPlot)
    """

    # 1) Aggregate data records from temperature_data
    records = []
    for station, feature_dict in temperature_data.items():
        for feature_name, segment_list in feature_dict.items():
            for segment in segment_list:
                # Skip empty segments
                if segment.size == 0:
                    continue

                # Compute multiple statistics for each segment
                mean_temp = np.mean(segment)
                std_temp = np.std(segment)
                min_temp = np.min(segment)
                max_temp = np.max(segment)
                median_temp = np.median(segment)
                iqr_temp = np.percentile(segment, 75) - np.percentile(segment, 25)

                records.append({
                    'Station': station,
                    'Feature': feature_name,
                    'MeanTemp': mean_temp,
                    'StdTemp': std_temp,
                    'MinTemp': min_temp,
                    'MaxTemp': max_temp,
                    'MedianTemp': median_temp,
                    'IQR': iqr_temp
                })

    if not records:
        print("No temperature data available to visualize.")
        return

    # 2) Create a DataFrame from the records
    df = pd.DataFrame(records)

    # 3) Specify which columns we want to include in the pairplot
    feature_cols = ['MeanTemp', 'StdTemp', 'MinTemp', 'MaxTemp', 'MedianTemp', 'IQR']

    # 4) Create a pairplot
    #    - hue='Feature' colors each point by the actual feature label.
    #    - diag_kind='kde' uses kernel density estimates on the diagonal (instead of histograms).
    g = sns.pairplot(
        df,
        vars=feature_cols,
        hue='Feature',
        diag_kind='kde',
        plot_kws={'alpha': 0.7, 's': 60},    # transparency & point size in off-diagonal
        corner=False                         # set True to only show lower triangle
    )

    # 5) Improve layout and show
    g.fig.suptitle("Pairwise Feature Visualization by Subway Segment Feature", y=1.02)
    plt.show()

In [38]:
pairwise_feature_visualization(temperature_data)

C:\Users\alpha\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
C:\Users\alpha\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1075: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)
C:\Users\alpha\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1075: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)
C:\Users\alpha\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1075: FutureWarning: When grouping with a len

In [39]:
def create_radar_charts(temperature_data):
    """
    For each station, aggregate the temperature statistics for each feature and plot a radar chart.
    The radar chart has 6 axes corresponding to:
        - Mean Temperature
        - Standard Deviation
        - Minimum Temperature
        - Maximum Temperature
        - Median Temperature
        - Interquartile Range (IQR)
    
    Each station's radar chart will have one line per feature (TRACKS, PLATFORM, CEILING, COLUMN),
    showing the (normalized) profile of that feature's thermal characteristics.
    
    Parameters:
      temperature_data : dict
          Nested dictionary:
             temperature_data[station][feature] = list of 1D numpy arrays of temperature values.
    """
    # 1) First, aggregate the data by computing average statistics for each station-feature pair.
    stats = ['Mean', 'Std', 'Min', 'Max', 'Median', 'IQR']
    aggregated = {}  # Structure: aggregated[station][feature] = dict of avg stats
    
    for station, feat_dict in temperature_data.items():
        aggregated[station] = {}
        for feature, segments in feat_dict.items():
            # Lists to accumulate stats from each segment for this station-feature.
            mean_vals = []
            std_vals = []
            min_vals = []
            max_vals = []
            median_vals = []
            iqr_vals = []
            for seg in segments:
                if seg.size == 0:
                    continue
                mean_vals.append(np.mean(seg))
                std_vals.append(np.std(seg))
                min_vals.append(np.min(seg))
                max_vals.append(np.max(seg))
                median_vals.append(np.median(seg))
                iqr_vals.append(np.percentile(seg, 75) - np.percentile(seg, 25))
            # Only include if we have data
            if len(mean_vals) > 0:
                aggregated[station][feature] = {
                    'Mean': np.mean(mean_vals),
                    'Std': np.mean(std_vals),
                    'Min': np.mean(min_vals),
                    'Max': np.mean(max_vals),
                    'Median': np.mean(median_vals),
                    'IQR': np.mean(iqr_vals)
                }
    
    # 2) Create a DataFrame of all aggregated station-feature stats to normalize across all records.
    records = []
    for station, feat_dict in aggregated.items():
        for feature, stats_dict in feat_dict.items():
            record = {'Station': station, 'Feature': feature}
            record.update(stats_dict)
            records.append(record)
    if not records:
        print("No aggregated data available.")
        return
    df = pd.DataFrame(records)
    
    # 3) Normalize the stats columns (so that each axis is comparable)
    stat_cols = stats  # ['Mean','Std','Min','Max','Median','IQR']
    df_norm = df.copy()
    for col in stat_cols:
        df_norm[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    
    # 4) Set up the radar chart axes.
    # Number of variables
    num_vars = len(stat_cols)
    # Compute angle for each axis (in radians)
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    # Complete the loop by appending the start angle to the end.
    angles += angles[:1]
    
    # 5) Create subplots for each station.
    stations = sorted(df_norm['Station'].unique())
    num_stations = len(stations)
    
    # For example, arrange in 2 rows if possible.
    ncols = 5 if num_stations >= 5 else num_stations
    nrows = int(np.ceil(num_stations / ncols))
    
    # Define a color mapping for features.
    feature_types = df_norm['Feature'].unique()
    colors = plt.cm.Set2(np.linspace(0, 1, len(feature_types)))
    color_map = dict(zip(feature_types, colors))
    
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*4, nrows*4), subplot_kw={'polar': True})
    axs = np.array(axs).flatten()  # flatten in case of multiple subplots
    
    for i, station in enumerate(stations):
        ax = axs[i]
        # Filter the normalized dataframe for this station.
        df_station = df_norm[df_norm['Station'] == station]
        
        # For each feature in this station, plot a line on the radar chart.
        for _, row in df_station.iterrows():
            values = [row[col] for col in stat_cols]
            # Close the loop for radar chart.
            values += values[:1]
            ax.plot(angles, values, label=row['Feature'], color=color_map[row['Feature']], linewidth=2)
            ax.fill(angles, values, alpha=0.15, color=color_map[row['Feature']])
        
        # Set the theta grid (one label per stat)
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(stat_cols, fontsize=9)
        ax.set_title(f"Station {int(station):02d}", fontsize=12, y=1.1)
        # Optionally set the radial limits.
        ax.set_ylim(0, 1)
    
    # Remove any unused subplots.
    for j in range(i+1, len(axs)):
        fig.delaxes(axs[j])
    
    # Add one common legend for all subplots.
    # Create custom handles.
    from matplotlib.lines import Line2D
    legend_handles = [Line2D([0], [0], color=color_map[ft], lw=2, label=ft) for ft in feature_types]
    fig.legend(handles=legend_handles, loc='upper right', bbox_to_anchor=(0.95, 0.95))
    
    plt.suptitle("Radar Charts of Thermal Profiles by Station and Feature", fontsize=16, y=0.98)
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])
    plt.show()

In [40]:
create_radar_charts(temperature_data)

In [41]:
# Directory paths
thermal_dir = "C:\\Users\\alpha\\Downloads\\161_YankeeStadium_THERMAL"
track_dir = "C:\\Users\\alpha\\Downloads\\161_YankeeStadium_TRACKS"
platform_dir = "C:\\Users\\alpha\\Downloads\\161_YankeeStadium_PLATFORM"
column_dir = "C:\\Users\\alpha\\Downloads\\161_YankeeStadium_COLUMNS"
ceiling_dir = "C:\\Users\\alpha\\Downloads\\161_YankeeStadium_CEILING"

In [42]:
def process_directories_2(thermal_dir, track_dir, platform_dir, ceiling_dir, column_dir):
    """
    Processes directories of thermal and segmentation mask images, returning a
    temperature_data dictionary keyed by date (e.g., "0702"), with each date
    mapping to a dict of features (TRACKS, PLATFORM, CEILING, COLUMN).

    The new naming convention is assumed to look like:
        - VISUAL070224BD161M01.png (for a mask)
        - FLIR070224BD161M01.jpg  (for the matching thermal image)
          ^^^^  ^^^^  ^^^^^^^^
          prefix (FLIR/VISUAL), date info, line+station+time+imageNo

    For example:
        "FLIR070224BD161A01.jpg"
         0.... 1234
               ^--- The next 4 chars might be "0702" for July 2
               then "24" for year, "BD161A01" for line/station/time

    We parse out the first 4 digits after the prefix as 'date_key' = "0702".

    Args:
        thermal_dir (str): directory path containing the FLIR thermal images
        track_dir   (str): directory path containing track masks
        platform_dir(str): directory path containing platform masks
        ceiling_dir (str): directory path containing ceiling masks
        column_dir  (str): directory path containing column masks

    Returns:
        dict: temperature_data with structure:
              {
                '0702': {  # date
                  'TRACKS': [ array_of_temps_for_segment1, array_of_temps_for_segment2, ... ],
                  'PLATFORM': [...],
                  'CEILING': [...],
                  'COLUMNS': [...]
                },
                '0703': { ... },
                ...
              }
    """
    from PIL import Image

    # A helper function to load mask images
    def load_mask_image(image_path):
        mask_image = Image.open(image_path).convert("L")  # Convert to grayscale
        mask_array = np.array(mask_image)
        binary_mask = (mask_array > 0).astype(np.uint8)
        return binary_mask

    # A helper function to extract temperature where mask == 1
    def extract_temp_for_class(flirobj, mask):
        therm = flirobj.get_thermal_np()
        therm_masked = np.ma.masked_where(mask != 1, therm)
        return therm_masked

    # Directory mapping for each feature
    feature_dirs = {
        'TRACKS': track_dir,
        'PLATFORM': platform_dir,
        'CEILING': ceiling_dir,
        'COLUMNS': column_dir
    }

    # We'll build a dictionary keyed by date string (e.g., "0702")
    temperature_data = {}

    # Process each feature directory
    for feature, dir_path in feature_dirs.items():
        print(f"Processing feature: {feature} from directory: {dir_path}")
        for file_name in os.listdir(dir_path):
            print(f"  - Processing file: {file_name}")

            # 1) Parse out the identifier (everything after the feature name)
            #    e.g., if file_name is "TRACKS070224BD161M01.png",
            #    then set_id might be "070224BD161M01.png".
            set_id = file_name.split(feature)[1]  # e.g., "070224BD161M01.png"

            # 2) Extract the first 4 digits from set_id as the date
            #    e.g. "0702" from "070224BD161M01.png"
            date_key = set_id[:4]  # "0702"

            # Ensure this date_key is in temperature_data
            if date_key not in temperature_data:
                temperature_data[date_key] = {f: [] for f in feature_dirs.keys()}

            # 3) Build the path to the mask
            mask_path = os.path.join(dir_path, file_name)
            mask = load_mask_image(mask_path)

            # 4) Build the matching thermal file name
            #    If the mask is "TRACKS070224BD161M01.png",
            #    then the thermal file is "FLIR070224BD161M01.jpg"
            #    i.e., we replace "TRACKS" with "FLIR" in the prefix.
            #    But we have set_id already, so we can do:
            #    "FLIR" + set_id, but replace extension with .jpg
            #    or assume .jpg is correct from the pattern.
            #    e.g. "FLIR070224BD161M01.jpg"
            base_no_ext = os.path.splitext(set_id)[0]  # "070224BD161M01"
            thermal_file_name = f"FLIR{base_no_ext}.jpg"
            thermal_path = os.path.join(thermal_dir, thermal_file_name)

            # 5) Try processing the thermal + mask
            try:
                flir = flirimageextractor.FlirImageExtractor(
                    "C:\\Users\\alpha\\exiftool-12.92_64\\exiftool.exe"
                )
                flir.process_image(thermal_path)

                temperature_masked = extract_temp_for_class(flir, mask)
                if temperature_masked is not None:
                    # Compressed form is a 1D array of valid (unmasked) temps
                    temperature_data[date_key][feature].append(temperature_masked.compressed())

                print(f"    - Processed {thermal_path} and {mask_path} successfully.")
            except Exception as e:
                print(f"    - Failed to process {thermal_path} or {mask_path}: {e}")

    return temperature_data

In [43]:
# Process directories and extract temperature data
temperature_data = process_directories_2(thermal_dir, track_dir, platform_dir, ceiling_dir, column_dir)

Processing feature: TRACKS from directory: C:\Users\alpha\Downloads\161_YankeeStadium_TRACKS
  - Processing file: TRACKS070324BD161M01.png
    - Processed C:\Users\alpha\Downloads\161_YankeeStadium_THERMAL\FLIR070324BD161M01.jpg and C:\Users\alpha\Downloads\161_YankeeStadium_TRACKS\TRACKS070324BD161M01.png successfully.
  - Processing file: TRACKS070324BD161M02.png
    - Processed C:\Users\alpha\Downloads\161_YankeeStadium_THERMAL\FLIR070324BD161M02.jpg and C:\Users\alpha\Downloads\161_YankeeStadium_TRACKS\TRACKS070324BD161M02.png successfully.
  - Processing file: TRACKS070324BD161M03.png
    - Processed C:\Users\alpha\Downloads\161_YankeeStadium_THERMAL\FLIR070324BD161M03.jpg and C:\Users\alpha\Downloads\161_YankeeStadium_TRACKS\TRACKS070324BD161M03.png successfully.
  - Processing file: TRACKS070924BD161M01.png
    - Processed C:\Users\alpha\Downloads\161_YankeeStadium_THERMAL\FLIR070924BD161M01.jpg and C:\Users\alpha\Downloads\161_YankeeStadium_TRACKS\TRACKS070924BD161M01.png succe

In [44]:
def create_date_series_violin_plot(temperature_data):
    """
    Creates a violin plot showing temperature distributions across multiple dates
    for a single station (e.g., 161 St-Yankee Stadium). The x-axis is 'Date', 
    the y-axis is 'Temperature', and the hue indicates the Feature type 
    (TRACKS, PLATFORM, CEILING, COLUMNS).

    Parameters
    ----------
    temperature_data : dict
        A dictionary with the structure:
            temperature_data[date][feature] = list of 1D numpy arrays
        Each 1D array contains temperature values from the segmented area.

    Returns
    -------
    None (displays the plot).
    """
    # 1) Flatten the data into a list of (date, feature, temperature_value)
    flattened_data = []
    for date, features in temperature_data.items():
        for feature, temp_lists in features.items():
            for temp_list in temp_lists:
                for temp in temp_list:
                    flattened_data.append((date, feature, temp))

    # 2) Convert to DataFrame
    df = pd.DataFrame(flattened_data, columns=['Date', 'Feature', 'Temperature'])

    # 3) Create the violin plot
    plt.figure(figsize=(14, 8))
    sns.violinplot(x='Date', y='Temperature', hue='Feature', data=df,
                   split=False, inner='quartile', dodge=True,
                   palette={'TRACKS': 'blue', 'PLATFORM': 'green',
                            'CEILING': 'orange', 'COLUMNS': 'purple'})

    # 4) Formatting
    plt.title('Time-Series Plot for Temperature Distributions by Date for 161 St-Yankee Station (BD)')
    plt.xlabel('Date')
    plt.ylabel('Temperature (°C)')
    plt.ylim(0, 44)
    plt.legend(title='Feature', loc='upper right')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [45]:
# Example usage (assuming you've populated temperature_data by date):
create_date_series_violin_plot(temperature_data)

In [46]:
def create_radar_charts_2(temperature_data):
    """
    For each date in the temperature_data dictionary (keyed by date string, e.g., "0702"),
    this function aggregates thermal statistics for each feature (TRACKS, PLATFORM, CEILING, COLUMN)
    by averaging over all segments, and then produces a radar chart for each date.
    
    The radar chart has six axes corresponding to:
      - Mean Temperature
      - Standard Deviation (Std)
      - Minimum Temperature (Min)
      - Maximum Temperature (Max)
      - Median Temperature
      - Interquartile Range (IQR)
    
    Each chart shows one line per feature (with a distinct color) representing the normalized
    profile of that feature's thermal characteristics on that date.
    
    Parameters
    ----------
    temperature_data : dict
        Dictionary with structure:
            temperature_data[date][feature] = list of 1D numpy arrays of temperature values.
    
    Returns
    -------
    None; displays the radar charts.
    """
    stats = ['Mean', 'Std', 'Min', 'Max', 'Median', 'IQR']
    aggregated = {}  # aggregated[date][feature] = dict of average stats
    
    # 1) Aggregate statistics for each date and feature.
    for date, feat_dict in temperature_data.items():
        aggregated[date] = {}
        for feature, segments in feat_dict.items():
            mean_vals = []
            std_vals = []
            min_vals = []
            max_vals = []
            median_vals = []
            iqr_vals = []
            for seg in segments:
                if seg.size == 0:
                    continue
                mean_vals.append(np.mean(seg))
                std_vals.append(np.std(seg))
                min_vals.append(np.min(seg))
                max_vals.append(np.max(seg))
                median_vals.append(np.median(seg))
                iqr_vals.append(np.percentile(seg, 75) - np.percentile(seg, 25))
            if len(mean_vals) > 0:
                aggregated[date][feature] = {
                    'Mean': np.mean(mean_vals),
                    'Std': np.mean(std_vals),
                    'Min': np.mean(min_vals),
                    'Max': np.mean(max_vals),
                    'Median': np.mean(median_vals),
                    'IQR': np.mean(iqr_vals)
                }
    
    # 2) Build a DataFrame where each row is a (date, feature) combination.
    records = []
    for date, feat_dict in aggregated.items():
        for feature, stats_dict in feat_dict.items():
            rec = {'Date': date, 'Feature': feature}
            rec.update(stats_dict)
            records.append(rec)
    
    if not records:
        print("No aggregated data available.")
        return
    
    df = pd.DataFrame(records)
    
    # 3) Normalize the statistics for fair comparison.
    stat_cols = stats
    df_norm = df.copy()
    for col in stat_cols:
        df_norm[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    
    # 4) Prepare radar chart axes.
    num_vars = len(stat_cols)
    # Angles (in radians) for each axis
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    # Complete the loop
    angles += angles[:1]
    
    # 5) Get unique dates (each will have its own radar chart)
    unique_dates = sorted(df_norm['Date'].unique())
    
    # Create a color mapping for the features.
    features = sorted(df_norm['Feature'].unique())
    cmap = plt.cm.get_cmap("Set2", len(features))
    feature_colors = {feature: cmap(i) for i, feature in enumerate(features)}
    
    # 6) Create one radar chart per date.
    n_dates = len(unique_dates)
    ncols = min(4, n_dates)
    nrows = int(np.ceil(n_dates / ncols))
    
    fig, axs = plt.subplots(nrows, ncols, figsize=(ncols * 4, nrows * 4), subplot_kw={'polar': True})
    axs = np.array(axs).flatten()  # flatten in case of grid
    for i, date in enumerate(unique_dates):
        ax = axs[i]
        df_date = df_norm[df_norm['Date'] == date]
        for _, row in df_date.iterrows():
            values = [row[col] for col in stat_cols]
            values += values[:1]  # Close the circle
            ax.plot(angles, values, label=row['Feature'], color=feature_colors[row['Feature']], linewidth=2)
            ax.fill(angles, values, color=feature_colors[row['Feature']], alpha=0.25)
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(stat_cols, fontsize=9)
        ax.set_title(f"Date: {date}", fontsize=12, y=1.1)
        ax.set_ylim(0, 1)
    
    # Remove any unused subplots.
    for j in range(i + 1, len(axs)):
        fig.delaxes(axs[j])
    
    # Add a single common legend.
    from matplotlib.lines import Line2D
    legend_handles = [Line2D([0], [0], color=feature_colors[f], lw=2, label=f) for f in features]
    fig.legend(handles=legend_handles, loc='upper right', bbox_to_anchor=(0.95, 0.95))
    
    plt.suptitle("Radar Charts of Thermal Profiles by Date and Feature for 161 St-Yankee Station", fontsize=16, y=0.98)
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])
    plt.show()

In [47]:
# Example usage:
# Assuming temperature_data (keyed by date) is populated using process_directories_2:
create_radar_charts_2(temperature_data)

C:\NAK-TEMP\ipykernel_33304\3287884678.py:92: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap("Set2", len(features))


In [ ]:
# Create function to process all of the images into a dictionary of sorts, which should be called `temperature_data`.
"""
    Processes directories of thermal and segmentation mask images, returning a
    temperature_data dictionary keyed by date (e.g., "0702"), with each date
    mapping to a dict of features (TRACKS, PLATFORM, CEILING, COLUMN).

    The new naming convention is assumed to look like:
        - VISUAL070224BD161M01.png (for a mask)
        - FLIR070224BD161M01.jpg  (for the matching thermal image)
          ^^^^  ^^^^  ^^^^^^^^
          prefix (FLIR/VISUAL), date info, line+station+time+imageNo

    For example:
        "FLIR070224BD161A01.jpg"
         0.... 1234
               ^--- The next 4 chars might be "0702" for July 2
               then "24" for year, "BD161A01" for line/station/time (morning, afternoon, or evening, M, A, or E).

    We parse out the first 4 digits after the prefix as 'date_key' = "0702".

    Args:
        thermal_dir (str): directory path containing the FLIR thermal images
        track_dir   (str): directory path containing track masks
        platform_dir(str): directory path containing platform masks
        ceiling_dir (str): directory path containing ceiling masks
        column_dir  (str): directory path containing column masks

    Returns:
        dict: temperature_data with structure:
              {
                '0702': {  # date
                  'TRACKS': [ array_of_temps_for_segment1, array_of_temps_for_segment2, ... ],
                  'PLATFORM': [...],
                  'CEILING': [...],
                  'COLUMNS': [...]
                },
                '0703': { ... },
                ...
              }
    """

def process_directories_final(thermal_dir, track_dir, platform_dir, ceiling_dir, column_dir)

In [ ]:
# Directory paths
thermal_dir = "C:\\Users\\alpha\\Downloads\\161_YankeeStadium_THERMAL"
track_dir = "C:\\Users\\alpha\\Downloads\\161_YankeeStadium_TRACKS"
platform_dir = "C:\\Users\\alpha\\Downloads\\161_YankeeStadium_PLATFORM"
column_dir = "C:\\Users\\alpha\\Downloads\\161_YankeeStadium_COLUMNS"
ceiling_dir = "C:\\Users\\alpha\\Downloads\\161_YankeeStadium_CEILING"

In [48]:
# Display the masked temperature data
plt.title("Thermal Image with Masked Temperature Data")
plt.imshow(pavement_mask, cmap='jet')
plt.colorbar(label='Temperature (°C)')
plt.axis('off')
plt.show()

In [49]:
## Check how well thermal and rgb registration is without manually correction
# You can see that the images do not line up and there is an offset even after 
# correcting for offset provided in file header
rgb_lowres, rgb_crop = extract_coarse_image(flir)

In [50]:
### Determine manual correction of Thermal and RGB registration 
offset, pts_temp, pts_rgb = manual_img_registration(flir)
print('X pixel offset is ' + str(offset[0]) + ' and Y pixel offset is ' + str(offset[1]))

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [51]:
rgb_lowres, rgb_crop = extract_coarse_image(flir, offset=offset, plot=1)

NameError: name 'offset' is not defined